In [253]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import time
from datetime import datetime
from scipy import integrate, optimize
import warnings
warnings.filterwarnings('ignore')

# ML libraries
import lightgbm as lgb
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, RidgeCV
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/countries-data/enriched_covid_19.csv
/kaggle/input/covid19-global-forecasting-week-4/submission.csv
/kaggle/input/covid19-global-forecasting-week-4/test.csv
/kaggle/input/covid19-global-forecasting-week-4/train.csv


In [254]:
counntries_df = pd.read_csv("/kaggle/input/countries-data/enriched_covid_19.csv")

In [373]:
train_df = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/train.csv")
test_df = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/test.csv")
submission = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/submission.csv")

In [374]:
train_df['Date'].max()

'2020-05-06'

In [375]:
test_df['Date'].min()

'2020-04-02'

In [257]:
train_df.describe()
#train_df.info()

,Id,ConfirmedCases,Fatalities
count,33178.000000,33178.000000,33178.000000
mean,17837.500000,2860.774218,186.124149
std,10300.654264,15961.579125,1538.811405
min,1.000000,0.000000,0.000000
25%,8919.250000,0.000000,0.000000
50%,17837.500000,12.000000,0.000000
75%,26755.750000,375.750000,5.000000
max,35674.000000,323978.000000,30076.000000


In [258]:
train_df['ConfirmedCases']=train_df['ConfirmedCases'].replace([-1], 0)
train_df['Fatalities']=train_df['Fatalities'].replace([-1], 0)

In [259]:
train_df[train_df['ConfirmedCases']==-1]

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities


In [260]:
train_df["key"]=train_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)
test_df["key"]=test_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)

#test_new=pd.merge(test,train, how="left", left_on=["key","Date"], right_on=["key","Date"] )
#train.to_csv(directory + "transfomed.csv")

target1="ConfirmedCases"
target2="Fatalities"
key="key"
def rate(frame, key, target, new_target_name="rate"):
   
    corrections = 0
    group_keys = frame[ key].values.tolist()
    target = frame[target].values.tolist()
    rate=[1.0 for k in range (len(target))]

    for i in range(1, len(group_keys) - 1):
        previous_group = group_keys[i - 1]
        current_group = group_keys[i]

        previous_value = target[i - 1]
        current_value = target[i]
         
        if current_group == previous_group:
                if previous_value!=0.0:
                     rate[i]=current_value/previous_value

                 
        rate[i] =max(1,rate[i] )#correct negative values

    frame[new_target_name] = np.array(rate)

In [261]:
all_data_train_test=train_df.copy()

In [262]:
all_data_train_test['mean_rate_case_last7']=0.00000000
all_data_train_test['mean_rate_case_last3']=0.00000000

all_data_train_test['mean_rate_case_last10']=0.00000000
all_data_train_test['mean_rate_fat_last10']=0.00000000

all_data_train_test['mean_rate_case_last5']=0.00000000
all_data_train_test['mean_rate_fat_last5']=0.00000000

all_data_train_test['mean_rate_case_last14']=0.00000000
all_data_train_test['mean_rate_fat_last14']=0.00000000

all_data_train_test['mean_rate_case_last20']=0.00000000
all_data_train_test['mean_rate_fat_last20']=0.00000000

all_data_train_test['mean_rate_case_each7']=0.00000000
all_data_train_test['mean_rate_case_each3']=0.00000000
all_data_train_test['mean_rate_case_each9']=0.00000000
all_data_train_test['mean_rate_case_each10']=0.00000000
all_data_train_test['mean_rate_case_each13']=0.00000000

all_data_train_test['mean_rate_fat_last7']=0.00000000
all_data_train_test['mean_rate_fat_last3']=0.00000000
all_data_train_test['mean_rate_fat_each7']=0.00000000
all_data_train_test['mean_rate_fat_each3']=0.00000000
all_data_train_test['mean_rate_fat_each9']=0.00000000
all_data_train_test['mean_rate_fat_each10']=0.00000000
all_data_train_test['mean_rate_fat_each13']=0.00000000

all_data_train_test['max_rate_case']=0.00000000
all_data_train_test['min_rate_case']=0.00000000
all_data_train_test['std_rate_case']=0.00000000
all_data_train_test['mode_rate_case']=0.00000000
all_data_train_test['range_rate_case']=0.00000000
all_data_train_test['max_to_min_rate_case']=0.00000000
all_data_train_test['max_rate_fat']=0.00000000
all_data_train_test['min_rate_fat']=0.00000000
all_data_train_test['std_rate_fat']=0.00000000
all_data_train_test['mode_rate_fat']=0.00000000
all_data_train_test['mean_rate_case']=0.00000000
all_data_train_test['mean_rate_fat']=0.00000000
all_data_train_test['range_rate_fat']=0.00000000       
all_data_train_test['max_to_min_rate_fat']=0.00000000   
all_data_train_test['cases_prev']=0  
all_data_train_test['fat_prev']=0   
all_data_train_test['mean_rate_fat_each10']=0.00000000

In [263]:
rate(all_data_train_test, key, target1, new_target_name="rate_" +target1)
rate(all_data_train_test, key, target2, new_target_name="rate_" +target2)
all_data_train_test.head()
all_data_train_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33178 entries, 0 to 33177
Data columns (total 47 columns):
Id                       33178 non-null int64
Province_State           14098 non-null object
Country_Region           33178 non-null object
Date                     33178 non-null object
ConfirmedCases           33178 non-null float64
Fatalities               33178 non-null float64
key                      33178 non-null object
mean_rate_case_last7     33178 non-null float64
mean_rate_case_last3     33178 non-null float64
mean_rate_case_last10    33178 non-null float64
mean_rate_fat_last10     33178 non-null float64
mean_rate_case_last5     33178 non-null float64
mean_rate_fat_last5      33178 non-null float64
mean_rate_case_last14    33178 non-null float64
mean_rate_fat_last14     33178 non-null float64
mean_rate_case_last20    33178 non-null float64
mean_rate_fat_last20     33178 non-null float64
mean_rate_case_each7     33178 non-null float64
mean_rate_case_each3     33178 no

In [264]:
def mean_rate_each(df, country, col,tar):
    target1=df[col][df['key']==country].values.tolist()
    mean_rate_case_each7=[0 for k in range (len(target1))]
    j=6
    for i in range(1, len(target1) - 1):
        if (i+j)<=(len(target1)):
            
            current_values1=[target1[i+j - 1],target1[i+j-2],target1[i+j -3],target1[i+j -4],target1[i+j -5],target1[i+j -6],target1[i+j -7]]
           
            mean_rate_case_each7[i+j-1]=np.mean(current_values1)
        else:
            break
                                         
    df[tar][df['key']==country]=np.array(mean_rate_case_each7)
def mean_rate_each3(df, country, col,tar):
    target1=df[col][df['key']==country].values.tolist()
    mean_rate_case_each3=[0 for k in range (len(target1))]
    j=2
    for i in range(1, len(target1) - 1):
        if (i+j)<=(len(target1)):
            
            current_values1=[target1[i+j - 1],target1[i+j-2],target1[i+j -3]]
           
            mean_rate_case_each3[i+j-1]=np.mean(current_values1)
        else:
            break
                                         
    df[tar][df['key']==country]=np.array(mean_rate_case_each3)            
    
def mean_rate_each10(df, country, col,tar):
    target1=df[col][df['key']==country].values.tolist()
    mean_rate_case_each10=[0 for k in range (len(target1))]
    j=9
    for i in range(1, len(target1) - 1):
        if (i+j)<=(len(target1)):
            
            current_values1=[target1[i+j - 1],target1[i+j-2],target1[i+j -3],target1[i+j -4],target1[i+j -5],target1[i+j -6],target1[i+j -7],target1[i+j -8],target1[i+j -9],target1[i+j -10]]
           
            mean_rate_case_each10[i+j-1]=np.mean(current_values1)
        else:
            break
                                         
    df[tar][df['key']==country]=np.array(mean_rate_case_each10)  
def mean_rate_each13(df, country, col,tar):
    target1=df[col][df['key']==country].values.tolist()
    mean_rate_case_each13=[0 for k in range (len(target1))]
    j=12
    for i in range(1, len(target1) - 1):
        if (i+j)<=(len(target1)):
            
            current_values1=[target1[i+j - 1],target1[i+j-2],target1[i+j -3],target1[i+j -4],target1[i+j -5],target1[i+j -6],target1[i+j -7],target1[i+j -8],target1[i+j -9],target1[i+j -10],target1[i+j -11],target1[i+j -12],target1[i+j -13]]
           
            mean_rate_case_each13[i+j-1]=np.mean(current_values1)
        else:
            break
                                         
    df[tar][df['key']==country]=np.array(mean_rate_case_each13)
    
def mean_rate_each9(df, country, col,tar):
    target1=df[col][df['key']==country].values.tolist()
    mean_rate_case_each9=[0 for k in range (len(target1))]
    j=8
    for i in range(1, len(target1) - 1):
        if (i+j)<=(len(target1)):
            
            current_values1=[target1[i+j - 1],target1[i+j-2],target1[i+j -3],target1[i+j -4],target1[i+j -5],target1[i+j -6],target1[i+j -7],target1[i+j -8],target1[i+j -9]]
           
            mean_rate_case_each9[i+j-1]=np.mean(current_values1)
        else:
            break
                                         
    df[tar][df['key']==country]=np.array(mean_rate_case_each9)

In [265]:
group_keys = all_data_train_test['key'].values.tolist()
for coc in group_keys:
    mean_rate_each(all_data_train_test, coc, col='rate_ConfirmedCases',tar='mean_rate_case_each7')
    mean_rate_each(all_data_train_test, coc, col='rate_Fatalities',tar='mean_rate_fat_each7')
    mean_rate_each3(all_data_train_test, coc, col='rate_ConfirmedCases',tar='mean_rate_case_each3')
    mean_rate_each3(all_data_train_test, coc, col='rate_Fatalities',tar='mean_rate_fat_each3')
    mean_rate_each13(all_data_train_test, coc, col='rate_ConfirmedCases',tar='mean_rate_case_each13')
    mean_rate_each13(all_data_train_test, coc, col='rate_Fatalities',tar='mean_rate_fat_each13')
    mean_rate_each9(all_data_train_test, coc, col='rate_ConfirmedCases',tar='mean_rate_case_each9')
    mean_rate_each9(all_data_train_test, coc, col='rate_Fatalities',tar='mean_rate_fat_each9')
    mean_rate_each10(all_data_train_test, coc, col='rate_ConfirmedCases',tar='mean_rate_case_each10')
    mean_rate_each10(all_data_train_test, coc, col='rate_Fatalities',tar='mean_rate_fat_each10')
    

KeyboardInterrupt: 

In [ ]:
group_keys = all_data_train_test['key'].values.tolist()
for coc in group_keys:
    

    all_data_train_test['mean_rate_case_each7']=all_data_train_test.rate_ConfirmedCases.rolling(7).mean() 
    all_data_train_test['mean_rate_case_each3']=all_data_train_test.rate_ConfirmedCases.rolling(3).mean() 
    all_data_train_test['mean_rate_case_each13']=all_data_train_test.rate_ConfirmedCases.rolling(13).mean() 
    all_data_train_test['mean_rate_case_each9']=all_data_train_test.rate_ConfirmedCases.rolling(9).mean() 
    all_data_train_test['mean_rate_case_each10']=all_data_train_test.rate_ConfirmedCases.rolling(10).mean() 

    all_data_train_test['mean_rate_fat_each7']=all_data_train_test.rate_Fatalities.rolling(7).mean() 
    all_data_train_test['mean_rate_fat_each3']=all_data_train_test.rate_Fatalities.rolling(3).mean() 
    all_data_train_test['mean_rate_fat_each13']=all_data_train_test.rate_Fatalities.rolling(13).mean() 
    all_data_train_test['mean_rate_fat_each9']=all_data_train_test.rate_Fatalities.rolling(9).mean() 
    all_data_train_test['mean_rate_fat_each10']=all_data_train_test.rate_Fatalities.rolling(10).mean() 


In [ ]:
all_data_train_test[['mean_rate_case_each7','rate_ConfirmedCases']][all_data_train_test['Country_Region']=='Italy'].head(40)

In [ ]:
all_data_train_test.info()

In [ ]:
all_data_train_test['Province_State'].fillna('NONE', inplace=True)
all_data_train_test['ConfirmedCases'].fillna(0, inplace=True)
all_data_train_test['Fatalities'].fillna(0, inplace=True)
all_data_train_test['Id'].fillna(-1, inplace=True)
#all_data_train_test['ForecastId'].fillna(-1, inplace=True)

In [ ]:
def infection_from_first_confirmed_cases(train_df,country):
    
    confirmed_cases = train_df[(train_df['Country_Region']==country_dict[country]) & train_df['ConfirmedCases']!=0].groupby(['Date']).agg({'ConfirmedCases':['sum']})
    fatalities_cases = train_df[(train_df['Country_Region']==country_dict[country]) & train_df['ConfirmedCases']!=0].groupby(['Date']).agg({'Fatalities':['sum']})
    total_cases = confirmed_cases.join(fatalities_cases)
    country_cases = [i for i in total_cases.ConfirmedCases['sum'].values]
    country_cases_filter = country_cases[0:70] 
       
    return country_cases_filter 

In [ ]:
#get the mean foe last seven vlaues for fatalities and confirmed cases
group_keys = all_data_train_test['key'].values.tolist()
for k in group_keys:
    all_data_train_test['mean_rate_case_last7'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].tail(7).mean()
    all_data_train_test['mean_rate_fat_last7'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k)&(all_data_train_test['Fatalities']!=0)].tail(7).mean()
    all_data_train_test['mean_rate_case_last3'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].tail(3).mean()
    all_data_train_test['mean_rate_fat_last3'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k)&(all_data_train_test['Fatalities']!=0)].tail(3).mean()
    all_data_train_test['std_rate_case'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].std()
    all_data_train_test['max_rate_case'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].max()
    all_data_train_test['min_rate_case'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].min()
    all_data_train_test['mode_rate_case'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].mode()
    all_data_train_test['mean_rate_case'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].mean()
    all_data_train_test['std_rate_fat'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k) & (all_data_train_test['Fatalities']!=0)].std()
    all_data_train_test['max_rate_fat'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k) & (all_data_train_test['Fatalities']!=0)].max()
    all_data_train_test['min_rate_fat'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k) & (all_data_train_test['Fatalities']!=0)].min()
    all_data_train_test['mode_rate_fat'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k) & (all_data_train_test['Fatalities']!=0)].mode()
    all_data_train_test['mean_rate_fat'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k) & (all_data_train_test['Fatalities']!=0)].mean()
   
    all_data_train_test['mean_rate_case_last10'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].tail(10).mean()
    all_data_train_test['mean_rate_fat_last10'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k)&(all_data_train_test['Fatalities']!=0)].tail(10).mean()
    all_data_train_test['mean_rate_case_last5'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].tail(5).mean()
    all_data_train_test['mean_rate_fat_last5'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k)&(all_data_train_test['Fatalities']!=0)].tail(5).mean()

    all_data_train_test['mean_rate_case_last14'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].tail(14).mean()
    all_data_train_test['mean_rate_fat_last14'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k)&(all_data_train_test['Fatalities']!=0)].tail(14).mean()
    all_data_train_test['mean_rate_case_last20'][all_data_train_test['key']==k]=all_data_train_test['rate_ConfirmedCases'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)].tail(20).mean()
    all_data_train_test['mean_rate_fat_last20'][all_data_train_test['key']==k]=all_data_train_test['rate_Fatalities'][(all_data_train_test['key']==k)&(all_data_train_test['Fatalities']!=0)].tail(20).mean()
    
    
    
   # all_data_train_test['max_to_min_rate_case'][all_data_train_test['key']==k]=(all_data_train_test['max_rate_case'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)])/(all_data_train_test['min_rate_case'][(all_data_train_test['key']==k) & (all_data_train_test['ConfirmedCases']!=0)])
    #all_data_train_test['max_to_min_rate_fat'][all_data_train_test['key']==k]=(all_data_train_test['max_rate_fat'][(all_data_train_test['key']==k) & (all_data_train_test['Fatalities']!=0)])/(all_data_train_test['min_rate_fat'][(all_data_train_test['key']==k) & (all_data_train_test['Fatalities']!=0)])
    
#all_data_train_test['cases_prev']=all_data_train_test['ConfirmedCases'].shift()
#all_data_train_test['fat_prev']=all_data_train_test['Fatalities'].shift()                   

all_data_train_test['max_to_min_rate_case']=all_data_train_test['max_rate_case']/all_data_train_test['min_rate_case']
all_data_train_test['max_to_min_rate_fat']=all_data_train_test['max_rate_fat']/all_data_train_test['min_rate_fat']

In [ ]:
all_data_train_test['ConfirmedCases'][all_data_train_test['ConfirmedCases']==-1]

In [ ]:
train_df['Date'].max()

In [ ]:
all_data_train_test['cases_prev'] = all_data_train_test.groupby("key")["ConfirmedCases"].shift()
all_data_train_test['fat_prev'] = all_data_train_test.groupby("key")["Fatalities"].shift()

In [ ]:
for lag in range(1, ((train_df['Date'].max()-train_df['Date'].min()).days)):            
    all_data_train_test[f"lag_{lag}_cc"] = all_data_train_test.groupby("key")["ConfirmedCases"].shift(lag)
    all_data_train_test[f"lag_{lag}_ft"] = all_data_train_test.groupby("key")["Fatalities"].shift(lag)
 

In [ ]:
all_data_train_test[['mean_rate_case_last7','ConfirmedCases','rate_ConfirmedCases']][all_data_train_test['Country_Region']=='Qatar']

In [ ]:

all_data_train_test[all_data_train_test['Country_Region']=='Qatar'].tail(50)

In [ ]:
counntries_df.rename(columns={'Country/Region':'Country_Region','Province/State':'Province_State'},inplace = True)
counntries_df["key"]=counntries_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)


In [ ]:
counntries_df.columns

In [ ]:
contries_data_columns=['Province_State', 'Country_Region', 'Lat', 'Long', 'Date',
       'ConfirmedCases', 'Fatalities', 'age_0-4', 'age_5-9', 'age_10-14','key',
       'age_15-19', 'age_20-24', 'age_25-29', 'age_30-34', 'age_35-39',
       'age_40-44', 'age_45-49', 'age_50-54', 'age_55-59', 'age_60-64',
       'age_65-69', 'age_70-74', 'age_75-79', 'age_80-84', 'age_85-89',
       'age_90-94', 'age_95-99', 'age_100+', 'total_pop', 'smokers_perc',
       'density', 'urbanpop', 'hospibed', 'lung', 'femalelung', 'malelung',
       'restrictions', 'quarantine', 'schools']

In [ ]:
grouped_countries=counntries_df.groupby(['Country_Region'], as_index=False).agg({"Lat": "max","Long": "max",'total_pop': "max",
                                                                                         'smokers_perc': "max",'density': "max",'urbanpop': "max",
                                                                                         'hospibed': "max",'lung': "max",'femalelung': "max",
                                                                                         'malelung': "max",'restrictions': "max",'quarantine': "max",
                                                                                         'schools': "max",'age_0-4': "max",'age_100+': "max",'age_5-9': "max",
                                                                                         'age_95-99': "max",'age_90-94': "max",'age_85-89': "max",
                                                                                         'age_80-84': "max",'age_75-79': "max",'age_70-74': "max",
                                                                                         'age_65-69': "max",'age_60-64': "max",'age_55-59': "max",
                                                                                         'age_50-54': "max",'age_45-49': "max",'age_40-44': "max",
                                                                                         'age_35-39': "max",'age_30-34': "max",'age_25-29': "max",
                                                                                         'age_20-24': "max",'age_15-19': "max",'age_10-14': "max",
                                                                                         })

In [ ]:
grouped_countries=counntries_df.groupby(['key'], as_index=False).agg({"Lat": "max","Long": "max",'total_pop': "max",
                                                                                         'smokers_perc': "max",'density': "max",'urbanpop': "max",
                                                                                         'hospibed': "max",'lung': "max",'femalelung': "max",
                                                                                         'malelung': "max",'restrictions': "max",'quarantine': "max",
                                                                                         'schools': "max",'age_0-4': "max",'age_100+': "max",'age_5-9': "max",
                                                                                         'age_95-99': "max",'age_90-94': "max",'age_85-89': "max",
                                                                                         'age_80-84': "max",'age_75-79': "max",'age_70-74': "max",
                                                                                         'age_65-69': "max",'age_60-64': "max",'age_55-59': "max",
                                                                                         'age_50-54': "max",'age_45-49': "max",'age_40-44': "max",
                                                                                         'age_35-39': "max",'age_30-34': "max",'age_25-29': "max",
                                                                                         'age_20-24': "max",'age_15-19': "max",'age_10-14': "max",
                                                                                         })

In [ ]:
merged_train_df = pd.merge(all_data_train_test,
                 grouped_countries[['Lat', 'Long',
        'age_0-4', 'age_5-9', 'age_10-14','key',
       'age_15-19', 'age_20-24', 'age_25-29', 'age_30-34', 'age_35-39',
       'age_40-44', 'age_45-49', 'age_50-54', 'age_55-59', 'age_60-64',
       'age_65-69', 'age_70-74', 'age_75-79', 'age_80-84', 'age_85-89',
       'age_90-94', 'age_95-99', 'age_100+', 'total_pop', 'smokers_perc',
       'density', 'urbanpop', 'hospibed', 'lung', 'femalelung', 'malelung',
       'restrictions', 'quarantine', 'schools']],
                 on='key',how='left')

In [ ]:
merged_train_df['Date']= pd.to_datetime(merged_train_df['Date']) 

In [ ]:
merged_train_df['confirmed_to_pop']=merged_train_df['ConfirmedCases']/(merged_train_df['total_pop']*1000)
merged_train_df['fat_to_pop']=merged_train_df['Fatalities']/(merged_train_df['total_pop']*1000)

In [ ]:
merged_train_df[['confirmed_to_pop','ConfirmedCases','total_pop']][merged_train_df['Country_Region']=='Saudi Arabia']

In [ ]:
merged_train_df[merged_train_df.isnull().any(axis=1)]

In [ ]:
merged_train_df['Province_State'].fillna('NONE', inplace=True)

In [ ]:
merged_train_df.fillna(0, inplace=True)

In [ ]:
merged_train_df.head()

In [ ]:
number_c = merged_train_df['Country_Region']
countries = (merged_train_df['Country_Region'])
country_dict = dict(zip(countries, number_c)) 
country_dict

In [ ]:
merged_train_df.columns

In [ ]:
merged_train_df['Date_num'] = pd.to_datetime(merged_train_df['Date'])
merged_train_df['Date_num'] = merged_train_df['Date_num'].dt.strftime("%m%d")
merged_train_df['Date_num']

In [ ]:
merged_test_df['Date_num'] = pd.to_datetime(merged_test_df['Date'])
merged_test_df['Date_num'] = merged_test_df['Date_num'].dt.strftime("%m%d")
merged_test_df['Date_num']

In [ ]:
merged_train_df['Date_num']=merged_train_df['Date_num'].astype('int')
merged_test_df['Date_num']=merged_test_df['Date_num'].astype('int')
merged_train_df.info()

## Combined

In [ ]:
merged_train_df.columns

In [ ]:
#'rate_ConfirmedCases','rate_Fatalities',

from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
merged_train_df[['mean_rate_case_last7', 'mean_rate_case_last3',
       'mean_rate_case_last10', 'mean_rate_fat_last10', 'mean_rate_case_last5',
       'mean_rate_fat_last5', 'mean_rate_case_last14', 'mean_rate_fat_last14',
       'mean_rate_case_last20', 'mean_rate_fat_last20', 'mean_rate_case_each7',
       'mean_rate_case_each3', 'mean_rate_case_each9', 'mean_rate_case_each10',
       'mean_rate_case_each13', 'mean_rate_fat_last7', 'mean_rate_fat_last3',
       'mean_rate_fat_each7', 'mean_rate_fat_each3', 'mean_rate_fat_each9',
        'mean_rate_fat_each13', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat',
       'max_to_min_rate_fat', 'cases_prev', 'fat_prev', 'rate_ConfirmedCases',
       'rate_Fatalities', 'mean_rate_fat_each10', 'age_0-4',
       'age_5-9', 'age_10-14', 'age_15-19', 'age_20-24', 'age_25-29',
       'age_30-34', 'age_35-39', 'age_40-44', 'age_45-49', 'age_50-54',
       'age_55-59', 'age_60-64', 'age_65-69', 'age_70-74', 'age_75-79',
       'age_80-84', 'age_85-89', 'age_90-94', 'age_95-99', 'age_100+',
       'total_pop', 'smokers_perc', 'density', 'urbanpop', 'hospibed', 'lung',
       'femalelung', 'malelung', 'restrictions', 'quarantine', 'schools']] = mms.fit_transform(merged_train_df[['mean_rate_case_last7', 'mean_rate_case_last3',
       'mean_rate_case_last10', 'mean_rate_fat_last10', 'mean_rate_case_last5',
       'mean_rate_fat_last5', 'mean_rate_case_last14', 'mean_rate_fat_last14',
       'mean_rate_case_last20', 'mean_rate_fat_last20', 'mean_rate_case_each7',
       'mean_rate_case_each3', 'mean_rate_case_each9', 'mean_rate_case_each10',
       'mean_rate_case_each13', 'mean_rate_fat_last7', 'mean_rate_fat_last3',
       'mean_rate_fat_each7', 'mean_rate_fat_each3', 'mean_rate_fat_each9',
       'mean_rate_fat_each13', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat',
       'max_to_min_rate_fat', 'cases_prev', 'fat_prev', 'rate_ConfirmedCases',
       'rate_Fatalities', 'mean_rate_fat_each10', 'age_0-4',
       'age_5-9', 'age_10-14', 'age_15-19', 'age_20-24', 'age_25-29',
       'age_30-34', 'age_35-39', 'age_40-44', 'age_45-49', 'age_50-54',
       'age_55-59', 'age_60-64', 'age_65-69', 'age_70-74', 'age_75-79',
       'age_80-84', 'age_85-89', 'age_90-94', 'age_95-99', 'age_100+',
       'total_pop', 'smokers_perc', 'density', 'urbanpop', 'hospibed', 'lung',
       'femalelung', 'malelung', 'restrictions', 'quarantine', 'schools']])

In [ ]:
train = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/train.csv')

In [ ]:
train.info()

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder

import xgboost as xgb

from tensorflow.keras.optimizers import Nadam
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import tensorflow.keras.layers as KL
from datetime import timedelta
import numpy as np
import pandas as pd


import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge

import datetime
import gc
from tqdm import tqdm
def get_cpmp_sub(save_oof=False, save_public_test=False):
    train = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/train.csv')
    train['Province_State'].fillna('', inplace=True)
    train['Date'] = pd.to_datetime(train['Date'])
    train['day'] = train.Date.dt.dayofyear
    #train = train[train.day <= 85]
    train['geo'] = ['_'.join(x) for x in zip(train['Country_Region'], train['Province_State'])]
    train.fillna(0, inplace=True)
    train['ConfirmedCases']=train['ConfirmedCases'].replace([-1], 0)
    train['Fatalities']=train['Fatalities'].replace([-1], 0)
    train

    test = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-4/test.csv')
    test['Province_State'].fillna('', inplace=True)
    test['Date'] = pd.to_datetime(test['Date'])
    test['day'] = test.Date.dt.dayofyear
    test['geo'] = ['_'.join(x) for x in zip(test['Country_Region'], test['Province_State'])]
    test.fillna(0, inplace=True)
    test

    day_min = train['day'].min()
    train['day'] -= day_min
    test['day'] -= day_min

    min_test_val_day = test.day.min()
    max_test_val_day = train.day.max()
    max_test_day = test.day.max()
    num_days = max_test_day + 1

    min_test_val_day, max_test_val_day, num_days

    train['ForecastId'] = -1
    test['Id'] = -1
    test['ConfirmedCases'] = 0
    test['Fatalities'] = 0

    debug = False

    data = pd.concat([train,
                      test[test.day > max_test_val_day][train.columns]
                     ]).reset_index(drop=True)
    if debug:
        data = data[data['geo'] >= 'France_'].reset_index(drop=True)
    #del train, test
    gc.collect()

    dates = data[data['geo'] == 'France_'].Date.values

    if 0:
        gr = data.groupby('geo')
        data['ConfirmedCases'] = gr.ConfirmedCases.transform('cummax')
        data['Fatalities'] = gr.Fatalities.transform('cummax')

    geo_data = data.pivot(index='geo', columns='day', values='ForecastId')
    num_geo = geo_data.shape[0]
    geo_data

    geo_id = {}
    for i,g in enumerate(geo_data.index):
        geo_id[g] = i


    ConfirmedCases = data.pivot(index='geo', columns='day', values='ConfirmedCases')
    Fatalities = data.pivot(index='geo', columns='day', values='Fatalities')

    if debug:
        cases = ConfirmedCases.values
        deaths = Fatalities.values
    else:
        cases = np.log1p(ConfirmedCases.values)
        deaths = np.log1p(Fatalities.values)


    def get_dataset(start_pred, num_train, lag_period): #63,5,14
        days = np.arange( start_pred - num_train + 1, start_pred + 1)#(59,1,64)
        lag_cases = np.vstack([cases[:, d - lag_period : d] for d in days])
        lag_deaths = np.vstack([deaths[:, d - lag_period : d] for d in days])
        target_cases = np.vstack([cases[:, d : d + 1] for d in days])
        target_deaths = np.vstack([deaths[:, d : d + 1] for d in days])
        geo_ids = np.vstack([geo_ids_base for d in days])
        country_ids = np.vstack([country_ids_base for d in days])
        return lag_cases, lag_deaths, target_cases, target_deaths, geo_ids, country_ids, days

    def update_valid_dataset(data, pred_death, pred_case):
        lag_cases, lag_deaths, target_cases, target_deaths, geo_ids, country_ids, days = data
        day = days[-1] + 1
        new_lag_cases = np.hstack([lag_cases[:, 1:], pred_case])
        new_lag_deaths = np.hstack([lag_deaths[:, 1:], pred_death]) 
        new_target_cases = cases[:, day:day+1]
        new_target_deaths = deaths[:, day:day+1] 
        new_geo_ids = geo_ids  
        new_country_ids = country_ids  
        new_days = 1 + days
        return new_lag_cases, new_lag_deaths, new_target_cases, new_target_deaths, new_geo_ids, new_country_ids, new_days

    def fit_eval(lr_death, lr_case, data, start_lag_death, end_lag_death, num_lag_case, fit, score):
        lag_cases, lag_deaths, target_cases, target_deaths, geo_ids, country_ids, days = data

        X_death = np.hstack([lag_cases[:, -start_lag_death:-end_lag_death], country_ids])
        X_death = np.hstack([lag_deaths[:, -num_lag_case:], country_ids])
        X_death = np.hstack([lag_cases[:, -start_lag_death:-end_lag_death], lag_deaths[:, -num_lag_case:], country_ids])
        y_death = target_deaths
        y_death_prev = lag_deaths[:, -1:]
        if fit:
            if 0:
                keep = (y_death > 0).ravel()
                X_death = X_death[keep]
                y_death = y_death[keep]
                y_death_prev = y_death_prev[keep]
            lr_death.fit(X_death, y_death)
        y_pred_death = lr_death.predict(X_death)
        y_pred_death = np.maximum(y_pred_death, y_death_prev)

        X_case = np.hstack([lag_cases[:, -num_lag_case:], geo_ids])
        X_case = lag_cases[:, -num_lag_case:]
        y_case = target_cases
        y_case_prev = lag_cases[:, -1:]
        if fit:
            lr_case.fit(X_case, y_case)
        y_pred_case = lr_case.predict(X_case)
        y_pred_case = np.maximum(y_pred_case, y_case_prev)

        if score:
            death_score = val_score(y_death, y_pred_death)
            case_score = val_score(y_case, y_pred_case)
        else:
            death_score = 0
            case_score = 0

        return death_score, case_score, y_pred_death, y_pred_case

    def train_model(train, valid, start_lag_death, end_lag_death, num_lag_case, num_val, score=True):
        alpha = 3
        lr_death = Ridge(alpha=alpha, fit_intercept=False)
        lr_case = Ridge(alpha=alpha, fit_intercept=True)

        (train_death_score, train_case_score, train_pred_death, train_pred_case,
        ) = fit_eval(lr_death, lr_case, train, start_lag_death, end_lag_death, num_lag_case, fit=True, score=score)

        death_scores = []
        case_scores = []

        death_pred = []
        case_pred = []

        for i in range(num_val):

            (valid_death_score, valid_case_score, valid_pred_death, valid_pred_case,
            ) = fit_eval(lr_death, lr_case, valid, start_lag_death, end_lag_death, num_lag_case, fit=False, score=score)

            death_scores.append(valid_death_score)
            case_scores.append(valid_case_score)
            death_pred.append(valid_pred_death)
            case_pred.append(valid_pred_case)

            if 0:
                print('val death: %0.3f' %  valid_death_score,
                      'val case: %0.3f' %  valid_case_score,
                      'val : %0.3f' %  np.mean([valid_death_score, valid_case_score]),
                      flush=True)
            valid = update_valid_dataset(valid, valid_pred_death, valid_pred_case)

        if score:
            death_scores = np.sqrt(np.mean([s**2 for s in death_scores]))
            case_scores = np.sqrt(np.mean([s**2 for s in case_scores]))
            if 0:
                print('train death: %0.3f' %  train_death_score,
                      'train case: %0.3f' %  train_case_score,
                      'val death: %0.3f' %  death_scores,
                      'val case: %0.3f' %  case_scores,
                      'val : %0.3f' % ( (death_scores + case_scores) / 2),
                      flush=True)
            else:
                print('%0.4f' %  case_scores,
                      ', %0.4f' %  death_scores,
                      '= %0.4f' % ( (death_scores + case_scores) / 2),
                      flush=True)
        death_pred = np.hstack(death_pred)
        case_pred = np.hstack(case_pred)
        return death_scores, case_scores, death_pred, case_pred

    countries = [g.split('_')[0] for g in geo_data.index]
    countries = pd.factorize(countries)[0]

    country_ids_base = countries.reshape((-1, 1))
    ohe = OneHotEncoder(sparse=False)
    country_ids_base = 0.2 * ohe.fit_transform(country_ids_base)
    country_ids_base.shape

    geo_ids_base = np.arange(num_geo).reshape((-1, 1))
    ohe = OneHotEncoder(sparse=False)
    geo_ids_base = 0.1 * ohe.fit_transform(geo_ids_base)
    geo_ids_base.shape

    def val_score(true, pred):
        pred = np.log1p(np.round(np.expm1(pred) - 0.2))
        return np.sqrt(mean_squared_error(true.ravel(), pred.ravel()))

    def val_score(true, pred):
        return np.sqrt(mean_squared_error(true.ravel(), pred.ravel()))

#14,6,5,14

    start_lag_death, end_lag_death = 14, 6,
    num_train = 7
    num_lag_case = 14
    lag_period = max(start_lag_death, num_lag_case)  #14

    def get_oof(start_val_delta=0):   
        start_val = min_test_val_day + start_val_delta
        last_train = start_val - 1
        num_val = max_test_val_day - start_val + 1
        print(dates[start_val], start_val, num_val)
        train_data = get_dataset(last_train, num_train, lag_period)
        valid_data = get_dataset(start_val, 1, lag_period)
        _, _, val_death_preds, val_case_preds = train_model(train_data, valid_data, 
                                                            start_lag_death, end_lag_death, num_lag_case, num_val)

        pred_deaths = Fatalities.iloc[:, start_val:start_val+num_val].copy()
        pred_deaths.iloc[:, :] = np.expm1(val_death_preds)
        pred_deaths = pred_deaths.stack().reset_index()
        pred_deaths.columns = ['geo', 'day', 'Fatalities']
        pred_deaths

        pred_cases = ConfirmedCases.iloc[:, start_val:start_val+num_val].copy()
        pred_cases.iloc[:, :] = np.expm1(val_case_preds)
        pred_cases = pred_cases.stack().reset_index()
        pred_cases.columns = ['geo', 'day', 'ConfirmedCases']
        pred_cases

        sub = train[['Date', 'Id', 'geo', 'day']]
        sub = sub.merge(pred_cases, how='left', on=['geo', 'day'])
        sub = sub.merge(pred_deaths, how='left', on=['geo', 'day'])
        #sub = sub.fillna(0)
        sub = sub[sub.day >= start_val]
        sub = sub[['Id', 'ConfirmedCases', 'Fatalities']].copy()
        return sub

#'2020-03-27', '2020-03-24', '2020-03-21', '2020-03-18'
#'2020-03-22', '2020-03-19', '2020-03-16', '2020-03-13'
    if save_oof:
        for start_val_delta, date in zip(range(3, -8, -3),
                                  ['2020-03-22', '2020-03-19', '2020-03-16', '2020-03-13']):
            print(date, end=' ')
            oof = get_oof(start_val_delta)
            oof.to_csv('../submissions/cpmp-%s.csv' % date, index=None)

    def get_sub(start_val_delta=0):   
        start_val = min_test_val_day + start_val_delta  #64
        last_train = start_val - 1   #63
        num_val = max_test_val_day - start_val + 1 #11
        print(dates[last_train], start_val, num_val)
        num_lag_case = 14  #14
        train_data = get_dataset(last_train, num_train, lag_period)#63,5,14
        valid_data = get_dataset(start_val, 1, lag_period) #64,1,14
        _, _, val_death_preds, val_case_preds = train_model(train_data, valid_data, 
                                                            start_lag_death, end_lag_death, num_lag_case, num_val)

        pred_deaths = Fatalities.iloc[:, start_val:start_val+num_val].copy()
        pred_deaths.iloc[:, :] = np.expm1(val_death_preds)
        pred_deaths = pred_deaths.stack().reset_index()
        pred_deaths.columns = ['geo', 'day', 'Fatalities']
        pred_deaths

        pred_cases = ConfirmedCases.iloc[:, start_val:start_val+num_val].copy()
        pred_cases.iloc[:, :] = np.expm1(val_case_preds)
        pred_cases = pred_cases.stack().reset_index()
        pred_cases.columns = ['geo', 'day', 'ConfirmedCases']
        pred_cases

        sub = test[['Date', 'ForecastId', 'geo', 'day']]
        sub = sub.merge(pred_cases, how='left', on=['geo', 'day'])
        sub = sub.merge(pred_deaths, how='left', on=['geo', 'day'])
        sub = sub.fillna(0)
        sub = sub[['ForecastId', 'ConfirmedCases', 'Fatalities']]
        return sub
        return sub


    known_test = train[['geo', 'day', 'ConfirmedCases', 'Fatalities']
              ].merge(test[['geo', 'day', 'ForecastId']], how='left', on=['geo', 'day'])
    known_test = known_test[['ForecastId', 'ConfirmedCases', 'Fatalities']][known_test.ForecastId.notnull()].copy()
    known_test

    unknow_test = test[test.day > max_test_val_day]
    unknow_test

    def get_final_sub():   
        start_val = max_test_val_day + 1
        last_train = start_val - 1
        num_val = max_test_day - start_val + 1
        print(dates[last_train], start_val, num_val)
        num_lag_case = num_val + 3
        train_data = get_dataset(last_train, num_train, lag_period)
        valid_data = get_dataset(start_val, 1, lag_period)
        (_, _, val_death_preds, val_case_preds
        ) = train_model(train_data, valid_data, start_lag_death, end_lag_death, num_lag_case, num_val, score=False)

        pred_deaths = Fatalities.iloc[:, start_val:start_val+num_val].copy()
        pred_deaths.iloc[:, :] = np.expm1(val_death_preds)
        pred_deaths = pred_deaths.stack().reset_index()
        pred_deaths.columns = ['geo', 'day', 'Fatalities']
        pred_deaths

        pred_cases = ConfirmedCases.iloc[:, start_val:start_val+num_val].copy()
        pred_cases.iloc[:, :] = np.expm1(val_case_preds)
        pred_cases = pred_cases.stack().reset_index()
        pred_cases.columns = ['geo', 'day', 'ConfirmedCases']
        pred_cases
        print(unknow_test.shape, pred_deaths.shape, pred_cases.shape)

        sub = unknow_test[['Date', 'ForecastId', 'geo', 'day']]
        sub = sub.merge(pred_cases, how='left', on=['geo', 'day'])
        sub = sub.merge(pred_deaths, how='left', on=['geo', 'day'])
        #sub = sub.fillna(0)
        sub = sub[['ForecastId', 'ConfirmedCases', 'Fatalities']]
        sub = pd.concat([known_test, sub])
        return sub

    if save_public_test:
        sub = get_sub()
    else:
        sub = get_final_sub()
    return sub

In [ ]:
merged_train_df.head()


# LGB

In [ ]:
from datetime import datetime as dt, timedelta
import lightgbm as lgb
from sklearn.metrics import mean_squared_error



In [ ]:
def calc_score(y_true, y_pred):
    y_true[y_true<0] = 0
    score = metrics.mean_squared_error(np.log(y_true.clip(0, 1e10)+1), np.log(y_pred[:]+1))**0.5
    return score

In [ ]:
SEED = 42
params = {'num_leaves': 8,
          'min_data_in_leaf': 5,  # 42,
          'objective': 'regression',
          'max_depth': 8,
          'learning_rate': 0.02,
          'boosting': 'gbdt',
          'bagging_freq': 5,  # 5
          'bagging_fraction': 0.8,  # 0.5,
          'feature_fraction': 0.8201,
          'bagging_seed': SEED,
          'reg_alpha': 1,  # 1.728910519108444,
          'reg_lambda': 4.9847051755586085,
          'random_state': SEED,
          'metric': 'mse',
          'verbosity': 100,
          'min_gain_to_split': 0.02,  # 0.01077313523861969,
          'min_child_weight': 5,  # 19.428902804238373,
          'num_threads': 6,
          }


In [ ]:
'mean_rate_case_last7', 'mean_rate_case_last3',
       'mean_rate_case_last10', 'mean_rate_fat_last10', 'mean_rate_case_last5',
       'mean_rate_fat_last5', 'mean_rate_case_last14', 'mean_rate_fat_last14',
       'mean_rate_case_last20', 'mean_rate_fat_last20', 'mean_rate_case_each7',
       'mean_rate_case_each3', 'mean_rate_case_each9', 'mean_rate_case_each10',
       'mean_rate_case_each13', 'mean_rate_fat_last7', 'mean_rate_fat_last3',
       'mean_rate_fat_each7', 'mean_rate_fat_each3', 'mean_rate_fat_each9',
       'mean_rate_fat_each20', 'mean_rate_fat_each13', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat',
       'max_to_min_rate_fat', 'cases_prev', 'fat_prev', 'rate_ConfirmedCases',
       'rate_Fatalities', 'mean_rate_fat_each10', 'age_0-4',
     
       'total_pop', 'smokers_perc', 'density'

In [ ]:
def get_nn_sub():
    df = pd.read_csv("../input/covid19-global-forecasting-week-4/train.csv")
    df['ConfirmedCases']=df['ConfirmedCases'].replace([-1], 0)
    df['Fatalities']=df['Fatalities'].replace([-1], 0)
    sub_df = pd.read_csv("../input/covid19-global-forecasting-week-4/test.csv")
    sub_df["key"]=sub_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)
    df["key"]=df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)
    features_columns = ['Id', 'Province_State', 'Country_Region', 'Date', 'ConfirmedCases',
       'Fatalities','mean_rate_case_last7', 'mean_rate_case_last3',
       'mean_rate_case_last10', 'mean_rate_fat_last10', 'mean_rate_case_last5',
       'mean_rate_fat_last5', 'mean_rate_case_last14', 'mean_rate_fat_last14',
       'mean_rate_case_last20', 'mean_rate_fat_last20', 'mean_rate_case_each7',
       'mean_rate_case_each3', 'mean_rate_case_each9', 'mean_rate_case_each10',
       'mean_rate_case_each13', 'mean_rate_fat_last7', 'mean_rate_fat_last3',
       'mean_rate_fat_each7', 'mean_rate_fat_each3', 'mean_rate_fat_each9',
       'mean_rate_fat_each13', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat','Lat','Long',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat',
       'max_to_min_rate_fat', 'cases_prev', 'fat_prev', 'rate_ConfirmedCases',
       'rate_Fatalities', 'mean_rate_fat_each10',  'total_pop', 'smokers_perc', 'density']

    '''coo_df = merged_train_df.groupby("Country_Region")[['mean_rate_case_last7', 'mean_rate_case_each7',
       'mean_rate_fat_last7', 'mean_rate_fat_each7', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat','cases_prev', 'fat_prev',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat','Lat','Long',
       'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities','total_pop', 'smokers_perc', 'density']].mean().reset_index()'''
    
    coo_df = merged_train_df.groupby("Country_Region")[['mean_rate_case_last7', 'mean_rate_case_last3',
       'mean_rate_case_last10', 'mean_rate_fat_last10', 'mean_rate_case_last5',
       'mean_rate_fat_last5', 'mean_rate_case_last14', 'mean_rate_fat_last14',
       'mean_rate_case_last20', 'mean_rate_fat_last20', 'mean_rate_case_each7',
       'mean_rate_case_each3', 'mean_rate_case_each9', 'mean_rate_case_each10',
       'mean_rate_case_each13', 'mean_rate_fat_last7', 'mean_rate_fat_last3',
       'mean_rate_fat_each7', 'mean_rate_fat_each3', 'mean_rate_fat_each9',
      'mean_rate_fat_each13', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat','Lat','Long',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat',
       'max_to_min_rate_fat', 'cases_prev', 'fat_prev', 'rate_ConfirmedCases',
       'rate_Fatalities', 'mean_rate_fat_each10', 'total_pop', 'smokers_perc', 'density']].mean().reset_index()
   # coo_df = coo_df[coo_df["Country_Region"].notnull()]

    loc_group = ["Province_State", "Country_Region"]
    #loc_group = ["key"]
   # merged_train_df['Date']= pd.to_datetime(merged_train_df['Date']) 
  #  merged_train_df["days"] = (merged_train_df["Date"] - pd.to_datetime("2020-01-01")).dt.days

    def preprocess(df):
        df["Date"] = df["Date"].astype("datetime64[ms]")
  #  df["days"] = (df["Date"] - pd.to_datetime("2020-01-01")).dt.days
    #    df["weekend"] = df["Date"].dt.dayofweek//5

        df = df.merge(coo_df, how="left", on="Country_Region")
        #df = df.merge(coo_df, how="left", on="key")
        df["Lat"] = (df["Lat"] // 30).astype(np.float32).fillna(0)
        df["Long"] = (df["Long"] // 60).astype(np.float32).fillna(0)

        for col in loc_group:
            df[col].fillna("none", inplace=True)
        return df
    merged_train_df2=pd.DataFrame(columns=features_columns)
    merged_train_df2=merged_train_df[features_columns].copy()
    #df = merged_train_df
    df = preprocess(df)
    sub_df = preprocess(sub_df)
   # coo_df = coo_df[coo_df["Country_Region"].notnull()]

   # merged_train_df['Date']= pd.to_datetime(merged_train_df['Date']) 
  #  merged_train_df["days"] = (merged_train_df["Date"] - pd.to_datetime("2020-01-01")).dt.days


 

    print(df.shape)

    TARGETS = ["ConfirmedCases", "Fatalities"]

    for col in TARGETS:
        df[col] = np.log1p(df[col])

    NUM_SHIFT = 7
#'1-smokers_perc','density'
#2-'smokers_perc','density','mean_rate_case_last7'
#3-'smokers_perc','density','mean_rate_case_last7','mean_rate_fat_last7'
#4-'smokers_perc','density','mean_rate_fat_last7'
#5-'mean_rate_case_last7','mean_rate_fat_last7'
#6- Lat and Long
#7-'mean_rate_case_last7', 'mean_rate_fat_last7','std_rate_case','std_rate_fat','density','Lat','Long'
#8----'mean_rate_case_last7', 'mean_rate_fat_last7','std_rate_case', 'std_rate_fat','mean_rate_case', 'mean_rate_fat','rate_ConfirmedCases','rate_Fatalities'
#9-'std_rate_case','std_rate_fat','mean_rate_case', 'mean_rate_fat','rate_ConfirmedCases','rate_Fatalities'
#10--------'mean_rate_case_last7', 'mean_rate_case_each7',  'mean_rate_fat_last7', 'mean_rate_fat_each7',   'rate_ConfirmedCases', 'rate_Fatalities'
#11    'mean_rate_case_last7', 'mean_rate_case_each7','std_rate_case','std_rate_fat', 'max_to_min_rate_case', 'max_to_min_rate_fat',       'mean_rate_fat_last7', 'mean_rate_fat_each7'
#12-----'mean_rate_case_last7', 'mean_rate_case_each7','mean_rate_case_last3','mean_rate_fat_last3', 'max_to_min_rate_case','max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities', 'mean_rate_case', 'mean_rate_fat','std_rate_fat','std_rate_case',              'mean_rate_fat_last7', 'mean_rate_fat_each7   
    # we have to tune
    
    features = ['mean_rate_case_last7', 'mean_rate_case_last3',
       'mean_rate_case_last10', 'mean_rate_fat_last10', 'mean_rate_case_last5',
       'mean_rate_fat_last5', 'mean_rate_case_last14', 'mean_rate_fat_last14',
       'mean_rate_case_last20', 'mean_rate_fat_last20', 'mean_rate_case_each7',
       'mean_rate_case_each3', 'mean_rate_case_each9', 'mean_rate_case_each10',
       'mean_rate_case_each13', 'mean_rate_fat_last7', 'mean_rate_fat_last3',
       'mean_rate_fat_each7', 'mean_rate_fat_each3', 'mean_rate_fat_each9',
       'mean_rate_fat_each13', 
      'rate_ConfirmedCases',
       'rate_Fatalities', 'mean_rate_fat_each10']
  

    for s in range(1, NUM_SHIFT+1):
        for col in TARGETS:
            df["prev_{}_{}".format(col, s)] = df.groupby(loc_group)[col].shift(s)
            features.append("prev_{}_{}".format(col, s))

    df = df[df["Date"] >= df["Date"].min() + timedelta(days=NUM_SHIFT)].copy()

    TEST_FIRST = sub_df["Date"].min() # pd.to_datetime("2020-03-13") #
    TEST_DAYS = (df["Date"].max() - TEST_FIRST).days + 1

    dev_df, test_df = df[df["Date"] < TEST_FIRST].copy(), df[df["Date"] >= TEST_FIRST].copy()

    def nn_block(input_layer, size, dropout_rate, activation):
        out_layer = KL.Dense(size, activation=None)(input_layer)
        #out_layer = KL.BatchNormalization()(out_layer)
        out_layer = KL.Activation(activation)(out_layer)
        out_layer = KL.Dropout(dropout_rate)(out_layer)
        return out_layer
    '''def get_model():
        inp = KL.Input(shape=(len(features),))

        hidden_layer = nn_block(inp, 256,0, "relu")
        hidden_layer = nn_block(hidden_layer,128, 0.0, "relu")
        gate_layer = nn_block(hidden_layer,64, 0.0, "sigmoid")
        hidden_layer = nn_block(hidden_layer,128, 0, "relu")
        hidden_layer = nn_block(hidden_layer,64, 0.1, "relu") #0.05
        hidden_layer = KL.multiply([hidden_layer, gate_layer])

        out = KL.Dense(len(TARGETS), activation="linear")(hidden_layer)

        model = tf.keras.models.Model(inputs=[inp], outputs=out)
        return model'''
    '''def get_model():
        inp = KL.Input(shape=(len(features),))

        hidden_layer = nn_block(inp, 128,0, "relu")
        hidden_layer = nn_block(hidden_layer,64, 0.0, "relu")
        gate_layer = nn_block(hidden_layer,32, 0.0, "sigmoid")

        hidden_layer = nn_block(hidden_layer,32, 0.2, "relu") #0.05
        hidden_layer = KL.multiply([hidden_layer, gate_layer])

        out = KL.Dense(len(TARGETS), activation="linear")(hidden_layer)

        model = tf.keras.models.Model(inputs=[inp], outputs=out)
        return model'''


    def get_model():
        inp = KL.Input(shape=(len(features),))

        hidden_layer = nn_block(inp, 64, 0.0, "relu")
        gate_layer = nn_block(hidden_layer, 32, 0.0, "sigmoid")
        hidden_layer = nn_block(hidden_layer, 32, 0, "relu")
       # hidden_layer = nn_block(hidden_layer, 64, 0, "relu")
        hidden_layer = KL.multiply([hidden_layer, gate_layer])

        out = KL.Dense(len(TARGETS), activation="linear")(hidden_layer)

        model = tf.keras.models.Model(inputs=[inp], outputs=out)
        return model
    '''def get_model():
        inp = KL.Input(shape=(len(features),))

        hidden_layer = nn_block(inp, 64, 0.0, "relu")#64
        gate_layer = nn_block(hidden_layer, 32, 0.0, "sigmoid")#32
        hidden_layer = nn_block(hidden_layer, 32, 0, "relu")
        hidden_layer = KL.multiply([hidden_layer, gate_layer])

        out = KL.Dense(len(TARGETS), activation="linear")(hidden_layer)

        model = tf.keras.models.Model(inputs=[inp], outputs=out)
        return model'''

    get_model().summary()

    def get_input(df):
        return [df[features]]

    NUM_MODELS = 10


    def train_models(df, save=False):
        models = []
        for i in range(NUM_MODELS):
            model = get_model()
            model.compile(loss="mean_squared_error", optimizer=Nadam(lr=1e-4))
            hist = model.fit(get_input(df), df[TARGETS],
                             batch_size=2048, epochs=500, verbose=0, shuffle=True)
            if save:
                model.save_weights("model{}.h5".format(i))
            models.append(model)
        return models

    models = train_models(dev_df)


    prev_targets = ['prev_ConfirmedCases_1', 'prev_Fatalities_1']

    def predict_one(df, models):
        pred = np.zeros((df.shape[0], 2))
        for model in models:
            pred += model.predict(get_input(df))/len(models)
        pred = np.maximum(pred, df[prev_targets].values)
        pred[:, 0] = np.log1p(np.expm1(pred[:, 0]) + 0.1)
        pred[:, 1] = np.log1p(np.expm1(pred[:, 1]) + 0.01)
        return np.clip(pred, None, 15)

    print([mean_squared_error(dev_df[TARGETS[i]], predict_one(dev_df, models)[:, i]) for i in range(len(TARGETS))])


    def rmse(y_true, y_pred):
        return np.sqrt(mean_squared_error(y_true, y_pred))

    def evaluate(df):
        error = 0
        for col in TARGETS:
            error += rmse(df[col].values, df["pred_{}".format(col)].values)
        return np.round(error/len(TARGETS), 5)


    def predict(test_df, first_day, num_days, models, val=False):
        temp_df = test_df.loc[test_df["Date"] == first_day].copy()
        y_pred = predict_one(temp_df, models)

        for i, col in enumerate(TARGETS):
            test_df["pred_{}".format(col)] = 0
            test_df.loc[test_df["Date"] == first_day, "pred_{}".format(col)] = y_pred[:, i]

        print(first_day, np.isnan(y_pred).sum(), y_pred.min(), y_pred.max())
        if val:
            print(evaluate(test_df[test_df["Date"] == first_day]))


        y_prevs = [None]*NUM_SHIFT

        for i in range(1, NUM_SHIFT):
            y_prevs[i] = temp_df[['prev_ConfirmedCases_{}'.format(i), 'prev_Fatalities_{}'.format(i)]].values

        for d in range(1, num_days):
            date = first_day + timedelta(days=d)
            print(date, np.isnan(y_pred).sum(), y_pred.min(), y_pred.max())

            temp_df = test_df.loc[test_df["Date"] == date].copy()
            temp_df[prev_targets] = y_pred
            for i in range(2, NUM_SHIFT+1):
                temp_df[['prev_ConfirmedCases_{}'.format(i), 'prev_Fatalities_{}'.format(i)]] = y_prevs[i-1]

            y_pred, y_prevs = predict_one(temp_df, models), [None, y_pred] + y_prevs[1:-1]


            for i, col in enumerate(TARGETS):
                test_df.loc[test_df["Date"] == date, "pred_{}".format(col)] = y_pred[:, i]

            if val:
                print(evaluate(test_df[test_df["Date"] == date]))

        return test_df

    test_df = predict(test_df, TEST_FIRST, TEST_DAYS, models, val=True)
    print(evaluate(test_df))

    for col in TARGETS:
        test_df[col] = np.expm1(test_df[col])
        test_df["pred_{}".format(col)] = np.expm1(test_df["pred_{}".format(col)])

    models = train_models(df, save=True)

    sub_df_public = sub_df[sub_df["Date"] <= df["Date"].max()].copy()
    sub_df_private = sub_df[sub_df["Date"] > df["Date"].max()].copy()

    pred_cols = ["pred_{}".format(col) for col in TARGETS]
    #sub_df_public = sub_df_public.merge(test_df[["Date"] + loc_group + pred_cols].rename(columns={col: col[5:] for col in pred_cols}), 
    #                                    how="left", on=["Date"] + loc_group)
    sub_df_public = sub_df_public.merge(test_df[["Date"] + loc_group + TARGETS], how="left", on=["Date"] + loc_group)

    SUB_FIRST = sub_df_private["Date"].min()
    SUB_DAYS = (sub_df_private["Date"].max() - sub_df_private["Date"].min()).days + 1

    sub_df_private = df.append(sub_df_private, sort=False)

    for s in range(1, NUM_SHIFT+1):
        for col in TARGETS:
            sub_df_private["prev_{}_{}".format(col, s)] = sub_df_private.groupby(loc_group)[col].shift(s)

    sub_df_private = sub_df_private[sub_df_private["Date"] >= SUB_FIRST].copy()

    sub_df_private = predict(sub_df_private, SUB_FIRST, SUB_DAYS, models)

    for col in TARGETS:
        sub_df_private[col] = np.expm1(sub_df_private["pred_{}".format(col)])

    sub_df = sub_df_public.append(sub_df_private, sort=False)
    sub_df["ForecastId"] = sub_df["ForecastId"].astype(np.int16)

    return sub_df[["ForecastId"] + TARGETS]

In [ ]:
sub1 = get_cpmp_sub()
sub1['ForecastId'] = sub1['ForecastId'].astype('int')

In [ ]:
sub2 = get_nn_sub()

In [ ]:
sub1.sort_values("ForecastId", inplace=True)
sub2.sort_values("ForecastId", inplace=True)

In [ ]:
from sklearn.metrics import mean_squared_error

TARGETS = ["ConfirmedCases", "Fatalities"]

[np.sqrt(mean_squared_error(np.log1p(sub1[t].values), np.log1p(sub2[t].values))) for t in TARGETS]

In [ ]:
sub_df = sub1.copy()
for t in TARGETS:
    sub_df[t] = np.expm1(np.log1p(sub1[t].values)*0.00+ np.log1p(sub2[t].values)*1).astype('int')
    
sub_df.to_csv("submission.csv", index=False)

In [370]:
Predicted_data = pd.merge(df_sub ,
                 test_df[['ForecastId','Province_State','Country_Region','Date']],
                 on='ForecastId',how='left')

In [371]:
target1="ConfirmedCases"
target2="Fatalities"
key="Country_Region"
def trend(frame, key, target, new_target_name="trend"):
   
    corrections = 0
    group_keys = frame[ key].values.tolist()
    target = frame[target].values.tolist()
    trend=[1.0 for k in range (len(target))]

    for i in range(1, len(group_keys) - 1):
        previous_group = group_keys[i - 1]
        current_group = group_keys[i]

        previous_value = target[i - 1]
        current_value = target[i]
         
        if current_group == previous_group:
                if previous_value!=0.0:
                     trend[i]=current_value-previous_value

                 
        trend[i] =max(1,trend[i] )#correct negative values

    frame[new_target_name] = np.array(trend)
trend(Predicted_data, key, target1, new_target_name="trend_" +target1)
trend(Predicted_data, key, target2, new_target_name="trend_" +target2)    

In [372]:
Predicted_data[Predicted_data['Country_Region']=='Qatar']


,ForecastId,ConfirmedCases,Fatalities,Province_State,Country_Region,Date,trend_ConfirmedCases,trend_Fatalities
8686,8687,949.000000,3.000000,NaN,Qatar,2020-04-02,1.0,1.0
8687,8688,1075.000000,3.000000,NaN,Qatar,2020-04-03,126.0,1.0
8688,8689,1325.000000,3.000000,NaN,Qatar,2020-04-04,250.0,1.0
8689,8690,1604.000000,4.000000,NaN,Qatar,2020-04-05,279.0,1.0
8690,8691,1832.000000,4.000000,NaN,Qatar,2020-04-06,228.0,1.0
8691,8692,2057.000000,6.000000,NaN,Qatar,2020-04-07,225.0,2.0
8692,8693,2210.000000,6.000000,NaN,Qatar,2020-04-08,153.0,1.0
8693,8694,2376.000000,6.000000,NaN,Qatar,2020-04-09,166.0,1.0
8694,8695,2512.000000,6.000000,NaN,Qatar,2020-04-10,136.0,1.0
8695,8696,2728.000000,6.000000,NaN,Qatar,2020-04-11,216.0,1.0


In [288]:
confirmed_total_date = Predicted_data.groupby(['Date']).agg({'trend_ConfirmedCases':'sum','ConfirmedCases':'sum'})
fatalities_total_date = Predicted_data.groupby(['Date']).agg({'trend_Fatalities':'sum','Fatalities':'sum'})
print(confirmed_total_date)
print(fatalities_total_date)

            trend_ConfirmedCases  ConfirmedCases
Date                                            
2020-04-02         336318.015414    1.013264e+06
2020-04-03          82744.000000    1.095922e+06
2020-04-04          80254.000000    1.176089e+06
2020-04-05          73825.000000    1.249834e+06
2020-04-06          71858.000000    1.321604e+06
2020-04-07          73852.000000    1.395367e+06
2020-04-08          85154.000000    1.480422e+06
2020-04-09          83751.000000    1.564019e+06
2020-04-10          93826.000000    1.657752e+06
2020-04-11          78213.000000    1.735848e+06
2020-04-12          99255.000000    1.834987e+06
2020-04-13          70131.000000    1.905015e+06
2020-04-14          70563.000000    1.975404e+06
2020-04-15          80023.000000    2.055327e+06
2020-04-16          96685.000000    2.151902e+06
2020-04-17          87855.000000    2.239544e+06
2020-04-18          77739.000000    2.317159e+06
2020-04-19          83930.000000    2.400991e+06
2020-04-20          

In [ ]:
confirmed_date = train_df.groupby(['Date']).agg({'ConfirmedCases':['sum']})
fatalities_date = train_df.groupby(['Date']).agg({'Fatalities':['sum']})
print(confirmed_date)
print(fatalities_date)

In [ ]:
train_df[train_df['Country_Region']=='Qatar'].tail(10)

In [ ]:
def countryplot(train_df,country):
    
    confirmed_total = train_df[train_df['Country_Region']==country_dict[country]].groupby(['Date']).agg({'ConfirmedCases':['sum']})
    fatalities_total = train_df[train_df['Country_Region']==country_dict[country]].groupby(['Date']).agg({'Fatalities':['sum']})
    total = confirmed_total.join(fatalities_total)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17,7))
    total.plot(ax=ax1)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=90)
    ax1.set_title(country + " Forecast Confirmed Cases and Fatalities ", size=13,)
    ax1.set_ylabel("Number of cases", size=13)
    ax1.set_xlabel("Date", size=13)
    #ax1.set_xticklabels("Date",rotation= 90)
    #plt.xticks(rotation=90)
    ax1.grid()
    fatalities_total.plot(ax=ax2, color='red')
    plt.grid()
    ax2.set_title(country+" Forecast Fatalities", size=13)
    ax2.set_ylabel("Number of cases", size=13)
    ax2.set_xlabel("Date", size=13)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
countrylist=['Qatar','Italy','Spain','United Arab Emirates','Egypt','Germany','France','Korea, South','Turkey','US','Saudi Arabia','United Kingdom']
for co in countrylist:
    countryplot(Predicted_data,co)
    #country='Italy'


In [ ]:
def countryplottotal(train_df):
      
    confirmed_total = train_df.groupby(['Date']).agg({'ConfirmedCases':['sum']})
    fatalities_total = train_df.groupby(['Date']).agg({'Fatalities':['sum']})
    total = confirmed_total.join(fatalities_total)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17,7))
    total.plot(ax=ax1)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=90)
    ax1.set_title("World Wide Forecast Confirmed Cases and Fatalities ", size=13,)
    ax1.set_ylabel("Number of cases", size=13)
    ax1.set_xlabel("Date", size=13)
    #ax1.set_xticklabels("Date",rotation= 90)
    #plt.xticks(rotation=90)
    ax1.grid()
    fatalities_total.plot(ax=ax2, color='red')
    plt.grid()
    ax2.set_title("World Wide Forecast Fatalities", size=13)
    ax2.set_ylabel("Number of cases", size=13)
    ax2.set_xlabel("Date", size=13)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
countryplottotal(Predicted_data)

In [ ]:
def countryplothist(train_df,country):
    
    confirmed_total = train_df[train_df['Country_Region']==country_dict[country]].groupby(['Date']).agg({'trend_ConfirmedCases':['sum']})
    fatalities_total = train_df[train_df['Country_Region']==country_dict[country]].groupby(['Date']).agg({'trend_Fatalities':['sum']})
    total = confirmed_total.join(fatalities_total)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17,7))
    total.plot(kind='bar',ax=ax1)
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=90)
    ax1.set_title(country + " Forecast Confirmed Cases and Fatalities ", size=13,)
    ax1.set_ylabel("Number of cases", size=13)
    ax1.set_xlabel("Date", size=13)
    #ax1.set_xticklabels("Date",rotation= 90)
    #plt.xticks(rotation=90)
    ax1.grid()
    fatalities_total.plot(kind='bar',ax=ax2, color='red')
    plt.grid()
    ax2.set_title(country+" Forecast Fatalities", size=13)
    ax2.set_ylabel("Number of cases", size=13)
    ax2.set_xlabel("Date", size=13)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
countrylist=['Qatar','Italy','Spain','United Arab Emirates','Egypt','Germany','France','Korea, South','Turkey','US','Saudi Arabia','United Kingdom']
for co in countrylist:
    countryplothist(Predicted_data,co)

In [299]:
df = pd.read_csv("../input/covid19-global-forecasting-week-4/train.csv")
df['ConfirmedCases']=df['ConfirmedCases'].replace([-1], 0)
df['Fatalities']=df['Fatalities'].replace([-1], 0)
sub_df = pd.read_csv("../input/covid19-global-forecasting-week-4/test.csv")
sub_df["key"]=sub_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)
df["key"]=df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)
features_columns = ['Id', 'Province_State', 'Country_Region', 'Date', 'ConfirmedCases',
       'Fatalities','mean_rate_case_last7', 'mean_rate_case_last3',
       'mean_rate_case_last10', 'mean_rate_fat_last10', 'mean_rate_case_last5',
       'mean_rate_fat_last5', 'mean_rate_case_last14', 'mean_rate_fat_last14',
       'mean_rate_case_last20', 'mean_rate_fat_last20', 'mean_rate_case_each7',
       'mean_rate_case_each3', 'mean_rate_case_each9', 'mean_rate_case_each10',
       'mean_rate_case_each13', 'mean_rate_fat_last7', 'mean_rate_fat_last3',
       'mean_rate_fat_each7', 'mean_rate_fat_each3', 'mean_rate_fat_each9',
       'mean_rate_fat_each13', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat','Lat','Long',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat',
       'max_to_min_rate_fat', 'cases_prev', 'fat_prev', 'rate_ConfirmedCases',
       'rate_Fatalities', 'mean_rate_fat_each10',  'total_pop', 'smokers_perc', 'density']

  
coo_df = merged_train_df.groupby("key")[['mean_rate_case_last7', 'mean_rate_case_last3',
       'mean_rate_case_last10', 'mean_rate_fat_last10', 'mean_rate_case_last5',
       'mean_rate_fat_last5', 'mean_rate_case_last14', 'mean_rate_fat_last14',
       'mean_rate_case_last20', 'mean_rate_fat_last20', 'mean_rate_case_each7',
       'mean_rate_case_each3', 'mean_rate_case_each9', 'mean_rate_case_each10',
       'mean_rate_case_each13', 'mean_rate_fat_last7', 'mean_rate_fat_last3',
       'mean_rate_fat_each7', 'mean_rate_fat_each3', 'mean_rate_fat_each9',
      'mean_rate_fat_each13', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat','Lat','Long',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat',
       'max_to_min_rate_fat', 'cases_prev', 'fat_prev', 'rate_ConfirmedCases',
       'rate_Fatalities', 'mean_rate_fat_each10', 'total_pop', 'smokers_perc', 'density']].mean().reset_index()
 

loc_group = ["Province_State", "Country_Region"]
    #loc_group = ["key"]
   # merged_train_df['Date']= pd.to_datetime(merged_train_df['Date']) 
  #  merged_train_df["days"] = (merged_train_df["Date"] - pd.to_datetime("2020-01-01")).dt.days

def preprocess(df):
    df["Date"] = df["Date"].astype("datetime64[ms]")

    df = df.merge(grouped_countries, how="left", on="key")
  
    df["Lat"] = (df["Lat"] // 30).astype(np.float32).fillna(0)
    df["Long"] = (df["Long"] // 60).astype(np.float32).fillna(0)

    for col in loc_group:
        
        df[col].fillna("none", inplace=True)
    return df


df = preprocess(df)
sub_df = preprocess(sub_df)

In [ ]:
sub_df.info()

In [ ]:
df.info()

In [300]:
#df = df.merge(sub_df, how="left", on="key")
#df_traintest=df.copy()
df_traintest = pd.concat([df, sub_df])
print(df.shape,sub_df.shape, df_traintest.shape)

(33178, 41) (13459, 39) (46637, 42)


In [302]:
df_traintest

,ConfirmedCases,Country_Region,Date,Fatalities,ForecastId,Id,Lat,Long,Province_State,age_0-4,...,hospibed,key,lung,malelung,quarantine,restrictions,schools,smokers_perc,total_pop,urbanpop
0,0.0,Afghanistan,2020-01-22,0.0,NaN,1.0,1.0,1.0,none,0.145717,...,0.5,nan_Afghanistan,37.62,39.33,0.0,0.0,0.0,21.389448,38928.341,25.0
1,0.0,Afghanistan,2020-01-23,0.0,NaN,2.0,1.0,1.0,none,0.145717,...,0.5,nan_Afghanistan,37.62,39.33,0.0,0.0,0.0,21.389448,38928.341,25.0
2,0.0,Afghanistan,2020-01-24,0.0,NaN,3.0,1.0,1.0,none,0.145717,...,0.5,nan_Afghanistan,37.62,39.33,0.0,0.0,0.0,21.389448,38928.341,25.0
3,0.0,Afghanistan,2020-01-25,0.0,NaN,4.0,1.0,1.0,none,0.145717,...,0.5,nan_Afghanistan,37.62,39.33,0.0,0.0,0.0,21.389448,38928.341,25.0
4,0.0,Afghanistan,2020-01-26,0.0,NaN,5.0,1.0,1.0,none,0.145717,...,0.5,nan_Afghanistan,37.62,39.33,0.0,0.0,0.0,21.389448,38928.341,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13454,NaN,Zimbabwe,2020-05-10,NaN,13455.0,NaN,0.0,0.0,none,NaN,...,NaN,nan_Zimbabwe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13455,NaN,Zimbabwe,2020-05-11,NaN,13456.0,NaN,0.0,0.0,none,NaN,...,NaN,nan_Zimbabwe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13456,NaN,Zimbabwe,2020-05-12,NaN,13457.0,NaN,0.0,0.0,none,NaN,...,NaN,nan_Zimbabwe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13457,NaN,Zimbabwe,2020-05-13,NaN,13458.0,NaN,0.0,0.0,none,NaN,...,NaN,nan_Zimbabwe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_traintest.info()

In [ ]:
df_traintest.tail()


In [303]:
def func(x):
    try:
        x_new = x['Country_Region'] + "_" + x['Province_State']
    except:
        x_new = x['Country_Region']
    return x_new

In [304]:
df_traintest['day'] = df_traintest['Date'].apply(lambda x: x.dayofyear).astype(np.int16)

In [305]:
df_traintest['place_id'] = df_traintest.apply(lambda x: func(x), axis=1)

In [306]:
places = np.sort(df_traintest['place_id'].unique())
print(len(places))

313


In [290]:
test_df.head()

,ForecastId,Province_State,Country_Region,Date,key
0,1,NaN,Afghanistan,2020-04-02,nan_Afghanistan
1,2,NaN,Afghanistan,2020-04-03,nan_Afghanistan
2,3,NaN,Afghanistan,2020-04-04,nan_Afghanistan
3,4,NaN,Afghanistan,2020-04-05,nan_Afghanistan
4,5,NaN,Afghanistan,2020-04-06,nan_Afghanistan


In [289]:
train_df.tail()

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,key
33173,35670,NaN,Zimbabwe,2020-05-02,34.0,4.0,nan_Zimbabwe
33174,35671,NaN,Zimbabwe,2020-05-03,34.0,4.0,nan_Zimbabwe
33175,35672,NaN,Zimbabwe,2020-05-04,34.0,4.0,nan_Zimbabwe
33176,35673,NaN,Zimbabwe,2020-05-05,34.0,4.0,nan_Zimbabwe
33177,35674,NaN,Zimbabwe,2020-05-06,34.0,4.0,nan_Zimbabwe


In [381]:
day_before_private = df_traintest['day'][pd.isna(df_traintest['ForecastId'])].max() # last day of train
day_before_public=day_before_private-35
day_before_valid=day_before_public-7


print(df_traintest['Date'][df_traintest['day']==day_before_valid].values[0]) # the7 days before the public
print(df_traintest['Date'][df_traintest['day']==day_before_public].values[0])#the day before the first date of the test
print(df_traintest['Date'][df_traintest['day']==day_before_private].values[0])#last date in the train

2020-03-25T00:00:00.000000000
2020-04-01T00:00:00.000000000
2020-05-06T00:00:00.000000000


In [308]:
df_traintest2 = df_traintest.copy()
df_traintest2['cases/day'] = 0
df_traintest2['fatal/day'] = 0
tmp_list = np.zeros(len(df_traintest2))
for place in places:
    tmp = df_traintest2['ConfirmedCases'][df_traintest2['place_id']==place].values
    tmp[1:] -= tmp[:-1]
    df_traintest2['cases/day'][df_traintest2['place_id']==place] = tmp
    tmp = df_traintest2['Fatalities'][df_traintest2['place_id']==place].values
    tmp[1:] -= tmp[:-1]
    df_traintest2['fatal/day'][df_traintest2['place_id']==place] = tmp
print(df_traintest2.shape)
df_traintest2[df_traintest2['place_id']=='US_New York'].tail(20)

(46637, 46)


,ConfirmedCases,Country_Region,Date,Fatalities,ForecastId,Id,Lat,Long,Province_State,age_0-4,...,quarantine,restrictions,schools,smokers_perc,total_pop,urbanpop,day,place_id,cases/day,fatal/day
11633,NaN,US,2020-04-25,NaN,11634.0,NaN,0.0,0.0,New York,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,116,US_New York,-9223372036854775808,-9223372036854775808
11634,NaN,US,2020-04-26,NaN,11635.0,NaN,0.0,0.0,New York,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,117,US_New York,-9223372036854775808,-9223372036854775808
11635,NaN,US,2020-04-27,NaN,11636.0,NaN,0.0,0.0,New York,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,118,US_New York,-9223372036854775808,-9223372036854775808
11636,NaN,US,2020-04-28,NaN,11637.0,NaN,0.0,0.0,New York,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,119,US_New York,-9223372036854775808,-9223372036854775808
11637,NaN,US,2020-04-29,NaN,11638.0,NaN,0.0,0.0,New York,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,120,US_New York,-9223372036854775808,-9223372036854775808
11638,NaN,US,2020-04-30,NaN,11639.0,NaN,0.0,0.0,New York,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,121,US_New York,-9223372036854775808,-9223372036854775808
11639,NaN,US,2020-05-01,NaN,11640.0,NaN,0.0,0.0,New York,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,122,US_New York,-9223372036854775808,-9223372036854775808
11640,NaN,US,2020-05-02,NaN,11641.0,NaN,0.0,0.0,New York,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,123,US_New York,-9223372036854775808,-9223372036854775808
11641,NaN,US,2020-05-03,NaN,11642.0,NaN,0.0,0.0,New York,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,124,US_New York,-9223372036854775808,-9223372036854775808
11642,NaN,US,2020-05-04,NaN,11643.0,NaN,0.0,0.0,New York,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,125,US_New York,-9223372036854775808,-9223372036854775808


In [309]:
def do_aggregation(df, col, mean_range):
    df_new = df.copy()
    col_new = '{}_({}-{})'.format(col, mean_range[0], mean_range[1])
    df_new[col_new] = 0
    tmp = df_new[col].rolling(mean_range[1]-mean_range[0]+1).mean()
    df_new[col_new][mean_range[0]:] = tmp[:-(mean_range[0])]
    df_new[col_new][pd.isna(df_new[col_new])] = 0
    return df_new[[col_new]].reset_index(drop=True)

def do_aggregations(df):
    df = pd.concat([df, do_aggregation(df, 'cases/day', [1,1]).reset_index(drop=True)], axis=1)
    df = pd.concat([df, do_aggregation(df, 'cases/day', [1,7]).reset_index(drop=True)], axis=1)
    df = pd.concat([df, do_aggregation(df, 'cases/day', [8,14]).reset_index(drop=True)], axis=1)
    df = pd.concat([df, do_aggregation(df, 'cases/day', [15,21]).reset_index(drop=True)], axis=1)
    df = pd.concat([df, do_aggregation(df, 'fatal/day', [1,1]).reset_index(drop=True)], axis=1)
    df = pd.concat([df, do_aggregation(df, 'fatal/day', [1,7]).reset_index(drop=True)], axis=1)
    df = pd.concat([df, do_aggregation(df, 'fatal/day', [8,14]).reset_index(drop=True)], axis=1)
    df = pd.concat([df, do_aggregation(df, 'fatal/day', [15,21]).reset_index(drop=True)], axis=1)
    for threshold in [1, 10, 100]:
        days_under_threshold = (df['ConfirmedCases']<threshold).sum()
        tmp = df['day'].values - 22 - days_under_threshold
        tmp[tmp<=0] = 0
        df['days_since_{}cases'.format(threshold)] = tmp
            
    for threshold in [1, 10, 100]:
        days_under_threshold = (df['Fatalities']<threshold).sum()
        tmp = df['day'].values - 22 - days_under_threshold
        tmp[tmp<=0] = 0
        df['days_since_{}fatal'.format(threshold)] = tmp
    
    # process China/Hubei
    if df['place_id'][0]=='China/Hubei':
        df['days_since_1cases'] += 35 # 2019/12/8
        df['days_since_10cases'] += 35-13 # 2019/12/8-2020/1/2 assume 2019/12/8+13
        df['days_since_100cases'] += 4 # 2020/1/18
        df['days_since_1fatal'] += 13 # 2020/1/9
    return df

In [310]:
df_traintest3 = []
for place in places[:]:
    df_tmp = df_traintest2[df_traintest2['place_id']==place].reset_index(drop=True)
    df_tmp = do_aggregations(df_tmp)
    df_traintest3.append(df_tmp)
df_traintest3 = pd.concat(df_traintest3).reset_index(drop=True)
df_traintest3[df_traintest3['place_id']=='China/Hubei'].head()

,ConfirmedCases,Country_Region,Date,Fatalities,ForecastId,Id,Lat,Long,Province_State,age_0-4,...,fatal/day_(1-1),fatal/day_(1-7),fatal/day_(8-14),fatal/day_(15-21),days_since_1cases,days_since_10cases,days_since_100cases,days_since_1fatal,days_since_10fatal,days_since_100fatal


In [311]:
def encode_label(df, col, freq_limit=0):
    df[col][pd.isna(df[col])] = 'nan'
    tmp = df[col].value_counts()
    cols = tmp.index.values
    freq = tmp.values
    num_cols = (freq>=freq_limit).sum()
    print("col: {}, num_cat: {}, num_reduced: {}".format(col, len(cols), num_cols))

    col_new = '{}_le'.format(col)
    df_new = pd.DataFrame(np.ones(len(df), np.int16)*(num_cols-1), columns=[col_new])
    for i, item in enumerate(cols[:num_cols]):
        df_new[col_new][df[col]==item] = i

    return df_new

def get_df_le(df, col_index, col_cat):
    df_new = df[[col_index]]
    for col in col_cat:
        df_tmp = encode_label(df, col)
        df_new = pd.concat([df_new, df_tmp], axis=1)
    return df_new

df_traintest3['id'] = np.arange(len(df_traintest3))
df_le = get_df_le(df_traintest3, 'id', ['Country_Region', 'Province_State'])
df_traintest4 = pd.merge(df_traintest3, df_le, on='id', how='left')

col: Country_Region, num_cat: 184, num_reduced: 184
col: Province_State, num_cat: 134, num_reduced: 134


In [312]:
df_traintest4['cases/day'] = df_traintest4['cases/day'].astype(np.float)
df_traintest4['fatal/day'] = df_traintest4['fatal/day'].astype(np.float)

In [313]:
df_traintest = df_traintest4.copy()

In [ ]:
#df_traintest['ForecastId'].fillna(0, inplace=True)

In [ ]:
df_traintest.info()

In [314]:
def calc_score(y_true, y_pred):
    y_true[y_true<0] = 0
    score = metrics.mean_squared_error(np.log(y_true.clip(0, 1e10)+1), np.log(y_pred[:]+1))**0.5
    return score
SEED = 42
params = {'num_leaves': 8,
          'min_data_in_leaf': 5,  # 42,
          'objective': 'regression',
          'max_depth': 8,
          'learning_rate': 0.02,
          'boosting': 'gbdt',
          'bagging_freq': 5,  # 5
          'bagging_fraction': 0.8,  # 0.5,
          'feature_fraction': 0.8201,
          'bagging_seed': SEED,
          'reg_alpha': 1,  # 1.728910519108444,
          'reg_lambda': 4.9847051755586085,
          'random_state': SEED,
          'metric': 'mse',
          'verbosity': 100,
          'min_gain_to_split': 0.02,  # 0.01077313523861969,
          'min_child_weight': 5,  # 19.428902804238373,
          'num_threads': 6,
          }

In [382]:
import os, gc, pickle, copy, datetime, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn import metrics
col_target = 'fatal/day'
col_var = [
'Lat', 'Long',
'smokers_perc', 'density',
    'cases/day_(1-1)', 
    'cases/day_(1-7)', 

    'fatal/day_(1-7)', 
    'fatal/day_(8-14)', 
    'fatal/day_(15-21)'
]
col_cat = []
df_train = df_traintest[(pd.isna(df_traintest['ForecastId'])) & (df_traintest['day']<=day_before_valid)]
df_valid = df_traintest[(pd.isna(df_traintest['ForecastId'])) & (day_before_valid<df_traintest['day'])& (df_traintest['day']<=(day_before_public))]
df_test = df_traintest[pd.isna(df_traintest['ForecastId'])==False]
X_train = df_train[col_var]
X_valid = df_valid[col_var]
y_train = np.log(df_train[col_target].values.clip(0, 1e10)+1)
y_valid = np.log(df_valid[col_target].values.clip(0, 1e10)+1)
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=col_cat)
valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=col_cat)
num_round = 15000
model = lgb.train(params, train_data, num_round, valid_sets=[train_data, valid_data],
                  verbose_eval=100,
                  early_stopping_rounds=150,)

best_itr = model.best_iteration

Training until validation scores don't improve for 150 rounds
[100]	training's l2: 0.0400577	valid_1's l2: 0.259248
[200]	training's l2: 0.0318574	valid_1's l2: 0.196848
[300]	training's l2: 0.0296271	valid_1's l2: 0.194332
[400]	training's l2: 0.0282063	valid_1's l2: 0.197171
Early stopping, best iteration is:
[260]	training's l2: 0.0303236	valid_1's l2: 0.193848


In [383]:
y_true = df_valid['fatal/day'].values
y_pred = np.exp(model.predict(X_valid))-1
score = calc_score(y_true, y_pred)
print("{:.6f}".format(score))

0.440282


In [317]:
tmp = pd.DataFrame()
tmp["feature"] = col_var
tmp["importance"] = model.feature_importance()
tmp = tmp.sort_values('importance', ascending=False)
tmp

,feature,importance
6,fatal/day_(1-7),572
5,cases/day_(1-7),270
4,cases/day_(1-1),112
8,fatal/day_(15-21),103
7,fatal/day_(8-14),94
3,density,37
2,smokers_perc,13
1,Long,3
0,Lat,0


In [384]:
df_train = df_traintest[(pd.isna(df_traintest['ForecastId'])) & (df_traintest['day']<=day_before_public)]
df_valid = df_traintest[(pd.isna(df_traintest['ForecastId'])) & (df_traintest['day']<=day_before_public)]
df_test = df_traintest[pd.isna(df_traintest['ForecastId'])==False]
X_train = df_train[col_var]
X_valid = df_valid[col_var]
y_train = np.log(df_train[col_target].values.clip(0, 1e10)+1)
y_valid = np.log(df_valid[col_target].values.clip(0, 1e10)+1)
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=col_cat)
valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=col_cat)
model = lgb.train(params, train_data, best_itr, valid_sets=[train_data, valid_data],
                  verbose_eval=100,
                  early_stopping_rounds=150,)

Training until validation scores don't improve for 150 rounds
[100]	training's l2: 0.0573425	valid_1's l2: 0.0573425
[200]	training's l2: 0.046612	valid_1's l2: 0.046612
Did not meet early stopping. Best iteration is:
[260]	training's l2: 0.0450911	valid_1's l2: 0.0450911


In [385]:
col_target2 = 'cases/day'
col_var2 = [
'Lat', 'Long',
'smokers_perc', 'density',
'days_since_10cases', #selected

    'cases/day_(1-1)', 
    'cases/day_(1-7)', 
    'cases/day_(8-14)',  
    'cases/day_(15-21)'
]
col_cat = []
df_train = df_traintest[(pd.isna(df_traintest['ForecastId'])) & (df_traintest['day']<=day_before_valid)]
df_valid = df_traintest[(pd.isna(df_traintest['ForecastId'])) & (day_before_valid<df_traintest['day']) & (df_traintest['day']<=day_before_public)]
X_train = df_train[col_var2]
X_valid = df_valid[col_var2]
y_train = np.log(df_train[col_target2].values.clip(0, 1e10)+1)
y_valid = np.log(df_valid[col_target2].values.clip(0, 1e10)+1)
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=col_cat)
valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=col_cat)
model2 = lgb.train(params, train_data, num_round, valid_sets=[train_data, valid_data],
                  verbose_eval=100,
                  early_stopping_rounds=150,)
best_itr2 = model2.best_iteration

Training until validation scores don't improve for 150 rounds
[100]	training's l2: 0.32426	valid_1's l2: 0.940213
[200]	training's l2: 0.274893	valid_1's l2: 0.714612
[300]	training's l2: 0.265383	valid_1's l2: 0.70225
[400]	training's l2: 0.259402	valid_1's l2: 0.703083
Early stopping, best iteration is:
[335]	training's l2: 0.263178	valid_1's l2: 0.700164


In [386]:
y_true = df_valid['cases/day'].values
y_pred = np.exp(model2.predict(X_valid))-1
score = calc_score(y_true, y_pred)
print("{:.6f}".format(score))

0.836758


In [ ]:
tmp = pd.DataFrame()
tmp["feature"] = col_var2
tmp["importance"] = model2.feature_importance()
tmp = tmp.sort_values('importance', ascending=False)
tmp

In [387]:
df_train = df_traintest[(pd.isna(df_traintest['ForecastId'])) & (df_traintest['day']<=day_before_public)]
df_valid = df_traintest[(pd.isna(df_traintest['ForecastId'])) & (df_traintest['day']<=day_before_public)]
X_train = df_train[col_var2]
X_valid = df_valid[col_var2]
y_train = np.log(df_train[col_target2].values.clip(0, 1e10)+1)
y_valid = np.log(df_valid[col_target2].values.clip(0, 1e10)+1)
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=col_cat)
valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=col_cat)
model2 = lgb.train(params, train_data, best_itr2, valid_sets=[train_data, valid_data],
                  verbose_eval=100,
                  early_stopping_rounds=150,)

Training until validation scores don't improve for 150 rounds
[100]	training's l2: 0.379998	valid_1's l2: 0.379998
[200]	training's l2: 0.317443	valid_1's l2: 0.317443
[300]	training's l2: 0.307682	valid_1's l2: 0.307682
Did not meet early stopping. Best iteration is:
[335]	training's l2: 0.305657	valid_1's l2: 0.305657


In [388]:
df_valid = df_traintest[(pd.isna(df_traintest['ForecastId'])) & ((day_before_public)<df_traintest['day'])]
df_valid

,ConfirmedCases,Country_Region,Date,Fatalities,ForecastId,Id,Lat,Long,Province_State,age_0-4,...,fatal/day_(15-21),days_since_1cases,days_since_10cases,days_since_100cases,days_since_1fatal,days_since_10fatal,days_since_100fatal,id,Country_Region_le,Province_State_le
71,273.0,Afghanistan,2020-04-02,6.0,NaN,72.0,1.0,1.0,none,0.145717,...,0.000000,38,19,6,11,0,0,71,168,0
72,281.0,Afghanistan,2020-04-03,6.0,NaN,73.0,1.0,1.0,none,0.145717,...,0.000000,39,20,7,12,0,0,72,168,0
73,299.0,Afghanistan,2020-04-04,7.0,NaN,74.0,1.0,1.0,none,0.145717,...,0.000000,40,21,8,13,0,0,73,168,0
74,349.0,Afghanistan,2020-04-05,7.0,NaN,75.0,1.0,1.0,none,0.145717,...,0.000000,41,22,9,14,0,0,74,168,0
75,367.0,Afghanistan,2020-04-06,11.0,NaN,76.0,1.0,1.0,none,0.145717,...,0.142857,42,23,10,15,0,0,75,168,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46589,34.0,Zimbabwe,2020-05-02,4.0,NaN,35670.0,0.0,0.0,none,NaN,...,0.000000,43,26,0,40,0,0,46589,79,0
46590,34.0,Zimbabwe,2020-05-03,4.0,NaN,35671.0,0.0,0.0,none,NaN,...,0.000000,44,27,0,41,0,0,46590,79,0
46591,34.0,Zimbabwe,2020-05-04,4.0,NaN,35672.0,0.0,0.0,none,NaN,...,0.000000,45,28,0,42,0,0,46591,79,0
46592,34.0,Zimbabwe,2020-05-05,4.0,NaN,35673.0,0.0,0.0,none,NaN,...,0.000000,46,29,0,43,0,0,46592,79,0


In [389]:
# train model to predict fatalities/day
df_train = df_traintest[(pd.isna(df_traintest['ForecastId'])) & (df_traintest['day']<=day_before_public)]
df_valid = df_traintest[(pd.isna(df_traintest['ForecastId'])) & ((day_before_public)<df_traintest['day'])]
df_test = df_traintest[pd.isna(df_traintest['ForecastId'])==False]
X_train = df_train[col_var]
X_valid = df_valid[col_var]
y_train = np.log(df_train[col_target].values.clip(0, 1e10)+1)
y_valid = np.log(df_valid[col_target].values.clip(0, 1e10)+1)
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=col_cat)
valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=col_cat)
num_round = 15000
model = lgb.train(params, train_data, num_round, valid_sets=[train_data, valid_data],
                  verbose_eval=100,
                  early_stopping_rounds=150,)

best_itr = model.best_iteration

Training until validation scores don't improve for 150 rounds
[100]	training's l2: 0.0573425	valid_1's l2: 0.292519
[200]	training's l2: 0.046612	valid_1's l2: 0.284098
Early stopping, best iteration is:
[144]	training's l2: 0.049573	valid_1's l2: 0.277878


In [390]:
# train with all data
df_train = df_traintest[(pd.isna(df_traintest['ForecastId']))]
df_valid = df_traintest[(pd.isna(df_traintest['ForecastId']))]
X_train = df_train[col_var]
X_valid = df_valid[col_var]
y_train = np.log(df_train[col_target].values.clip(0, 1e10)+1)
y_valid = np.log(df_valid[col_target].values.clip(0, 1e10)+1)
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=col_cat)
valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=col_cat)
model_pri = lgb.train(params, train_data, best_itr, valid_sets=[train_data, valid_data],
                  verbose_eval=100,
                  early_stopping_rounds=150,)

Training until validation scores don't improve for 150 rounds
[100]	training's l2: 0.134761	valid_1's l2: 0.134761
Did not meet early stopping. Best iteration is:
[144]	training's l2: 0.113892	valid_1's l2: 0.113892


In [391]:
df_train = df_traintest[(pd.isna(df_traintest['ForecastId'])) & (df_traintest['day']<=day_before_public)]
df_valid = df_traintest[(pd.isna(df_traintest['ForecastId'])) & ((day_before_public-1)<df_traintest['day'])]
X_train = df_train[col_var2]
X_valid = df_valid[col_var2]
y_train = np.log(df_train[col_target2].values.clip(0, 1e10)+1)
y_valid = np.log(df_valid[col_target2].values.clip(0, 1e10)+1)
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=col_cat)
valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=col_cat)
model2 = lgb.train(params, train_data, num_round, valid_sets=[train_data, valid_data],
                  verbose_eval=100,
                  early_stopping_rounds=150,)
best_itr2 = model2.best_iteration

Training until validation scores don't improve for 150 rounds
[100]	training's l2: 0.379998	valid_1's l2: 0.937846
[200]	training's l2: 0.317443	valid_1's l2: 0.819472
[300]	training's l2: 0.307682	valid_1's l2: 0.822586
Early stopping, best iteration is:
[218]	training's l2: 0.314904	valid_1's l2: 0.818699


In [392]:
# train with all data
df_train = df_traintest[(pd.isna(df_traintest['ForecastId'])) & (df_traintest['day']<=day_before_public)]
df_valid = df_traintest[(pd.isna(df_traintest['ForecastId'])) & (df_traintest['day']<=day_before_public)]
X_train = df_train[col_var2]
X_valid = df_valid[col_var2]
y_train = np.log(df_train[col_target2].values.clip(0, 1e10)+1)
y_valid = np.log(df_valid[col_target2].values.clip(0, 1e10)+1)
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=col_cat)
valid_data = lgb.Dataset(X_valid, label=y_valid, categorical_feature=col_cat)
model2_pri = lgb.train(params, train_data, best_itr2, valid_sets=[train_data, valid_data],
                  verbose_eval=100,
                  early_stopping_rounds=150,)

Training until validation scores don't improve for 150 rounds
[100]	training's l2: 0.379998	valid_1's l2: 0.379998
[200]	training's l2: 0.317443	valid_1's l2: 0.317443
Did not meet early stopping. Best iteration is:
[218]	training's l2: 0.314904	valid_1's l2: 0.314904


In [393]:
df_tmp = df_traintest[
    ((df_traintest['day']<=day_before_public)  & (pd.isna(df_traintest['ForecastId'])))
    | ((day_before_public<df_traintest['day']) & (pd.isna(df_traintest['ForecastId'])==False))].reset_index(drop=True)

df_traintest9 = []
for i, place in enumerate(places[:]):
    df_tmp2 = df_tmp[df_tmp['place_id']==place].reset_index(drop=True)
   # df_tmp2 = do_aggregations(df_tmp2)
    df_traintest9.append(df_tmp2)
df_traintest9 = pd.concat(df_traintest9).reset_index(drop=True)
df_traintest9[df_traintest9['day']>day_before_public-2].head()

,ConfirmedCases,Country_Region,Date,Fatalities,ForecastId,Id,Lat,Long,Province_State,age_0-4,...,fatal/day_(15-21),days_since_1cases,days_since_10cases,days_since_100cases,days_since_1fatal,days_since_10fatal,days_since_100fatal,id,Country_Region_le,Province_State_le
69,174.0,Afghanistan,2020-03-31,4.0,NaN,70.0,1.0,1.0,none,0.145717,...,0.000000,36,17,4,9,0,0,69,168,0
70,237.0,Afghanistan,2020-04-01,4.0,NaN,71.0,1.0,1.0,none,0.145717,...,0.000000,37,18,5,10,0,0,70,168,0
71,NaN,Afghanistan,2020-04-02,NaN,1.0,NaN,1.0,1.0,none,0.145717,...,2.142857,38,19,6,11,0,0,106,168,0
72,NaN,Afghanistan,2020-04-03,NaN,2.0,NaN,1.0,1.0,none,0.145717,...,1.714286,39,20,7,12,0,0,107,168,0
73,NaN,Afghanistan,2020-04-04,NaN,3.0,NaN,1.0,1.0,none,0.145717,...,1.857143,40,21,8,13,0,0,108,168,0


In [394]:
# remove overlap for private LB prediction
df_tmp = df_traintest[
    ((df_traintest['day']<=day_before_private)  & (pd.isna(df_traintest['ForecastId'])))
    | ((day_before_private<df_traintest['day']) & (pd.isna(df_traintest['ForecastId'])==False))].reset_index(drop=True)

df_traintest10 = []
for i, place in enumerate(places[:]):
    df_tmp2 = df_tmp[df_tmp['place_id']==place].reset_index(drop=True)

    df_traintest10.append(df_tmp2)
df_traintest10 = pd.concat(df_traintest10).reset_index(drop=True)
df_traintest10[df_traintest10['day']>day_before_private-2].head()

,ConfirmedCases,Country_Region,Date,Fatalities,ForecastId,Id,Lat,Long,Province_State,age_0-4,...,fatal/day_(15-21),days_since_1cases,days_since_10cases,days_since_100cases,days_since_1fatal,days_since_10fatal,days_since_100fatal,id,Country_Region_le,Province_State_le
104,3224.0,Afghanistan,2020-05-05,95.0,NaN,105.0,1.0,1.0,none,0.145717,...,2.142857e+00,71,52,39,44,29,0,104,168,0
105,3392.0,Afghanistan,2020-05-06,104.0,NaN,106.0,1.0,1.0,none,0.145717,...,1.857143e+00,72,53,40,45,30,0,105,168,0
106,NaN,Afghanistan,2020-05-07,NaN,36.0,NaN,1.0,1.0,none,0.145717,...,-9.223372e+18,73,54,41,46,31,1,141,168,0
107,NaN,Afghanistan,2020-05-08,NaN,37.0,NaN,1.0,1.0,none,0.145717,...,-9.223372e+18,74,55,42,47,32,2,142,168,0
108,NaN,Afghanistan,2020-05-09,NaN,38.0,NaN,1.0,1.0,none,0.145717,...,-9.223372e+18,75,56,43,48,33,3,143,168,0


In [395]:
# predict test data in public
# predict the cases and fatatilites one day at a time and use the predicts as next day's feature recursively.
df_preds = []
for i, place in enumerate(places[:]):
    df_interest = copy.deepcopy(df_traintest9[df_traintest9['place_id']==place].reset_index(drop=True))
    df_interest['cases/day'][(pd.isna(df_interest['ForecastId']))==False] = -1
    df_interest['fatal/day'][(pd.isna(df_interest['ForecastId']))==False] = -1
    len_known = (df_interest['day']<=day_before_public).sum()
    len_unknown = (day_before_public<df_interest['day']).sum()
    for j in range(len_unknown): # use predicted cases and fatal for next days' prediction
        X_valid = df_interest[col_var].iloc[j+len_known]
        X_valid2 = df_interest[col_var2].iloc[j+len_known]
        pred_f = model.predict(X_valid)
        pred_c = model2.predict(X_valid2)
        pred_c = (np.exp(pred_c)-1).clip(0, 1e10)
        pred_f = (np.exp(pred_f)-1).clip(0, 1e10)
        df_interest['fatal/day'][j+len_known] = pred_f
        df_interest['cases/day'][j+len_known] = pred_c
        df_interest['Fatalities'][j+len_known] = df_interest['Fatalities'][j+len_known-1] + pred_f
        df_interest['ConfirmedCases'][j+len_known] = df_interest['ConfirmedCases'][j+len_known-1] + pred_c
#         print(df_interest['ConfirmedCases'][j+len_known-1], df_interest['ConfirmedCases'][j+len_known], pred_c)
 
     
    if (i+1)%10==0:
        print("{:3d}/{}  {}, len known: {}, len unknown: {}".format(i+1, len(places), place, len_known, len_unknown), df_interest.shape)
    df_interest['fatal_pred'] = np.cumsum(df_interest['fatal/day'].values)
    df_interest['cases_pred'] = np.cumsum(df_interest['cases/day'].values)
    df_preds.append(df_interest)
df_preds = pd.concat(df_preds)

 10/313  Australia_New South Wales, len known: 71, len unknown: 43 (114, 63)
 20/313  Bahrain_none, len known: 71, len unknown: 43 (114, 63)
 30/313  Botswana_none, len known: 71, len unknown: 43 (114, 63)
 40/313  Canada_Alberta, len known: 71, len unknown: 43 (114, 63)
 50/313  Canada_Saskatchewan, len known: 71, len unknown: 43 (114, 63)
 60/313  China_Guangdong, len known: 71, len unknown: 43 (114, 63)
 70/313  China_Inner Mongolia, len known: 71, len unknown: 43 (114, 63)
 80/313  China_Shanghai, len known: 71, len unknown: 43 (114, 63)
 90/313  Congo (Kinshasa)_none, len known: 71, len unknown: 43 (114, 63)
100/313  Diamond Princess_none, len known: 71, len unknown: 43 (114, 63)
110/313  Eswatini_none, len known: 71, len unknown: 43 (114, 63)
120/313  France_Reunion, len known: 71, len unknown: 43 (114, 63)
130/313  Greece_none, len known: 71, len unknown: 43 (114, 63)
140/313  Iceland_none, len known: 71, len unknown: 43 (114, 63)
150/313  Jordan_none, len known: 71, len unknown

In [396]:
# predict test data in public
df_preds_pri = []
for i, place in enumerate(places[:]):
    df_interest = copy.deepcopy(df_traintest10[df_traintest10['place_id']==place].reset_index(drop=True))
    df_interest['cases/day'][(pd.isna(df_interest['ForecastId']))==False] = -1
    df_interest['fatal/day'][(pd.isna(df_interest['ForecastId']))==False] = -1
    len_known = (df_interest['day']<=day_before_private).sum()
    len_unknown = (day_before_private<df_interest['day']).sum()
    for j in range(len_unknown): # use predicted cases and fatal for next days' prediction
        X_valid = df_interest[col_var].iloc[j+len_known]
        X_valid2 = df_interest[col_var2].iloc[j+len_known]
        pred_f = model_pri.predict(X_valid)
        pred_c = model2_pri.predict(X_valid2)
        pred_c = (np.exp(pred_c)-1).clip(0, 1e10)
        pred_f = (np.exp(pred_f)-1).clip(0, 1e10)
        df_interest['fatal/day'][j+len_known] = pred_f
        df_interest['cases/day'][j+len_known] = pred_c
        df_interest['Fatalities'][j+len_known] = df_interest['Fatalities'][j+len_known-1] + pred_f
        df_interest['ConfirmedCases'][j+len_known] = df_interest['ConfirmedCases'][j+len_known-1] + pred_c

    if (i+1)%10==0:
        print("{:3d}/{}  {}, len known: {}, len unknown: {}".format(i+1, len(places), place, len_known, len_unknown), df_interest.shape)
    df_interest['fatal_pred'] = np.cumsum(df_interest['fatal/day'].values)
    df_interest['cases_pred'] = np.cumsum(df_interest['cases/day'].values)
    df_preds_pri.append(df_interest)
df_preds_pri = pd.concat(df_preds_pri)

 10/313  Australia_New South Wales, len known: 106, len unknown: 8 (114, 63)
 20/313  Bahrain_none, len known: 106, len unknown: 8 (114, 63)
 30/313  Botswana_none, len known: 106, len unknown: 8 (114, 63)
 40/313  Canada_Alberta, len known: 106, len unknown: 8 (114, 63)
 50/313  Canada_Saskatchewan, len known: 106, len unknown: 8 (114, 63)
 60/313  China_Guangdong, len known: 106, len unknown: 8 (114, 63)
 70/313  China_Inner Mongolia, len known: 106, len unknown: 8 (114, 63)
 80/313  China_Shanghai, len known: 106, len unknown: 8 (114, 63)
 90/313  Congo (Kinshasa)_none, len known: 106, len unknown: 8 (114, 63)
100/313  Diamond Princess_none, len known: 106, len unknown: 8 (114, 63)
110/313  Eswatini_none, len known: 106, len unknown: 8 (114, 63)
120/313  France_Reunion, len known: 106, len unknown: 8 (114, 63)
130/313  Greece_none, len known: 106, len unknown: 8 (114, 63)
140/313  Iceland_none, len known: 106, len unknown: 8 (114, 63)
150/313  Jordan_none, len known: 106, len unknow

In [397]:
places_sort = df_traintest10[['place_id', 'ConfirmedCases']][df_traintest10['day']==day_before_private]
places_sort = places_sort.sort_values('ConfirmedCases', ascending=False).reset_index(drop=True)['place_id'].values
print(len(places_sort))
places_sort[:5]

313


array(['US_New York', 'Spain_none', 'Italy_none', 'United Kingdom_none',
       'France_none'], dtype=object)

In [ ]:
print("Fatalities / Public")
plt.figure(figsize=(30,30))
for i in range(30):
    plt.subplot(5,6,i+1)
    idx = i * 10
    df_interest = df_preds[df_preds['place_id']==places_sort[idx]].reset_index(drop=True)
    tmp = df_interest['fatal/day'].values
    tmp = np.cumsum(tmp)
    sns.lineplot(x=df_interest['day'], y=tmp, label='pred')
    df_interest2 = df_traintest10[(df_traintest10['place_id']==places_sort[idx]) & (df_traintest10['day']<=day_before_private)].reset_index(drop=True)
    sns.lineplot(x=df_interest2['day'].values, y=df_interest2['Fatalities'].values, label='true')
    plt.title(places_sort[idx])
plt.show()

In [ ]:
print("Confirmed Cases / Public")
plt.figure(figsize=(30,30))
for i in range(30):
    plt.subplot(5,6,i+1)
    idx = i * 10
    df_interest = df_preds[df_preds['place_id']==places_sort[idx]].reset_index(drop=True)
    tmp = df_interest['cases/day'].values
    tmp = np.cumsum(tmp)
    sns.lineplot(x=df_interest['day'], y=tmp, label='pred')
    df_interest2 = df_traintest10[(df_traintest10['place_id']==places_sort[idx]) & (df_traintest10['day']<=day_before_private)].reset_index(drop=True)
    sns.lineplot(x=df_interest2['day'].values, y=df_interest2['ConfirmedCases'].values, label='true')
    plt.title(places_sort[idx])
plt.show()

In [ ]:
print("Fatalities / Private")
plt.figure(figsize=(30,30))
for i in range(30):
    plt.subplot(5,6,i+1)
    idx = i * 10
    df_interest = df_preds_pri[df_preds_pri['place_id']==places_sort[idx]].reset_index(drop=True)
    tmp = df_interest['fatal/day'].values
    tmp = np.cumsum(tmp)
    sns.lineplot(x=df_interest['day'], y=tmp, label='pred')
    df_interest2 = df_traintest10[(df_traintest10['place_id']==places_sort[idx]) & (df_traintest10['day']<=day_before_private)].reset_index(drop=True)
    sns.lineplot(x=df_interest2['day'].values, y=df_interest2['Fatalities'].values, label='true')
    plt.title(places_sort[idx])
plt.show()

In [ ]:
print("ConfirmedCases / Private")
plt.figure(figsize=(30,30))
for i in range(30):
    plt.subplot(5,6,i+1)
    idx = i * 10
    df_interest = df_preds_pri[df_preds_pri['place_id']==places_sort[idx]].reset_index(drop=True)
    tmp = df_interest['cases/day'].values
    tmp = np.cumsum(tmp)
    sns.lineplot(x=df_interest['day'], y=tmp, label='pred')
    df_interest2 = df_traintest10[(df_traintest10['place_id']==places_sort[idx]) & (df_traintest10['day']<=day_before_private)].reset_index(drop=True)
    sns.lineplot(x=df_interest2['day'].values, y=df_interest2['ConfirmedCases'].values, label='true')
    plt.title(places_sort[idx])
plt.show()

In [365]:
# merge 2 preds
df_preds[df_preds['day']>day_before_private] = df_preds_pri[df_preds['day']>day_before_private]
#df_preds[df_preds['day']>day_before_private] 

,ConfirmedCases,Country_Region,Date,Fatalities,ForecastId,Id,Lat,Long,Province_State,age_0-4,...,days_since_10cases,days_since_100cases,days_since_1fatal,days_since_10fatal,days_since_100fatal,id,Country_Region_le,Province_State_le,fatal_pred,cases_pred
106,1942.777458,Afghanistan,2020-05-07,60.176042,36.0,NaN,1.0,1.0,none,0.145717,...,54,41,46,31,1,141,168,0,60.176042,1942.777458
107,1943.251483,Afghanistan,2020-05-08,60.198047,37.0,NaN,1.0,1.0,none,0.145717,...,55,42,47,32,2,142,168,0,60.198047,1943.251483
108,1943.725508,Afghanistan,2020-05-09,60.220052,38.0,NaN,1.0,1.0,none,0.145717,...,56,43,48,33,3,143,168,0,60.220052,1943.725508
109,1944.199533,Afghanistan,2020-05-10,60.242058,39.0,NaN,1.0,1.0,none,0.145717,...,57,44,49,34,4,144,168,0,60.242058,1944.199533
110,1944.673558,Afghanistan,2020-05-11,60.264063,40.0,NaN,1.0,1.0,none,0.145717,...,58,45,50,35,5,145,168,0,60.264063,1944.673558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,35.847530,Zimbabwe,2020-05-10,4.242058,13455.0,NaN,0.0,0.0,none,NaN,...,34,3,48,3,3,46632,79,0,4.242058,35.847530
110,36.219094,Zimbabwe,2020-05-11,4.264063,13456.0,NaN,0.0,0.0,none,NaN,...,35,4,49,4,4,46633,79,0,4.264063,36.219094
111,36.590658,Zimbabwe,2020-05-12,4.286068,13457.0,NaN,0.0,0.0,none,NaN,...,36,5,50,5,5,46634,79,0,4.286068,36.590658
112,36.978669,Zimbabwe,2020-05-13,4.308073,13458.0,NaN,0.0,0.0,none,NaN,...,37,6,51,6,6,46635,79,0,4.308073,36.978669


In [366]:
df_preds.to_csv("df_preds.csv", index=None)

In [367]:
df_sub = pd.read_csv("../input/covid19-global-forecasting-week-4/submission.csv")
print(len(df_sub))
df_sub.head()

13459


,ForecastId,ConfirmedCases,Fatalities
0,1,1,1
1,2,1,1
2,3,1,1
3,4,1,1
4,5,1,1


In [368]:
df_sub = pd.merge(df_sub, df_traintest[['ForecastId', 'place_id', 'day']])
df_sub = pd.merge(df_sub, df_preds[['place_id', 'day', 'cases_pred', 'fatal_pred']], on=['place_id', 'day',], how='left')
df_sub.head(10)

,ForecastId,ConfirmedCases,Fatalities,place_id,day,cases_pred,fatal_pred
0,1,1,1,Afghanistan_none,93,273.0,6.0
1,2,1,1,Afghanistan_none,94,281.0,6.0
2,3,1,1,Afghanistan_none,95,299.0,7.0
3,4,1,1,Afghanistan_none,96,349.0,7.0
4,5,1,1,Afghanistan_none,97,367.0,11.0
5,6,1,1,Afghanistan_none,98,423.0,14.0
6,7,1,1,Afghanistan_none,99,444.0,14.0
7,8,1,1,Afghanistan_none,100,484.0,15.0
8,9,1,1,Afghanistan_none,101,521.0,15.0
9,10,1,1,Afghanistan_none,102,555.0,18.0


In [369]:
df_sub['ConfirmedCases'] = df_sub['cases_pred']
df_sub['Fatalities'] = df_sub['fatal_pred']
df_sub = df_sub[['ForecastId', 'ConfirmedCases', 'Fatalities']]
df_sub.to_csv("submission.csv", index=None)
df_sub.head(10)

,ForecastId,ConfirmedCases,Fatalities
0,1,273.0,6.0
1,2,281.0,6.0
2,3,299.0,7.0
3,4,349.0,7.0
4,5,367.0,11.0
5,6,423.0,14.0
6,7,444.0,14.0
7,8,484.0,15.0
8,9,521.0,15.0
9,10,555.0,18.0


In [ ]:
featureslist

In [ ]:
def func(x):
    try:
        x_new = x['Country_Region'] + "_" + x['Province_State']
    except:
        x_new = x['Country_Region']
    return x_new

In [ ]:
train_df = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/train.csv")
test_df = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/test.csv")
submission = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/submission.csv")

train_df['ConfirmedCases']=train_df['ConfirmedCases'].replace([-1], 0)
train_df['Fatalities']=train_df['Fatalities'].replace([-1], 0)
train_df['key'] = train_df.apply(lambda x: func(x), axis=1)
#train_df["key"]=train_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "" + str(row[1]),axis=1)

#train_df['cases_prev']=train_df.groupby("key")["ConfirmedCases"].shift()
#train_df['fat_prev']=train_df.groupby("key")["Fatalities"].shift()

train_df['cases_prev'] = train_df.groupby("key")["ConfirmedCases"].shift()
train_df['fat_prev'] = train_df.groupby("key")["Fatalities"].shift()

sub_df = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/test.csv")

#sub_df["key"]=sub_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "" + str(row[1]),axis=1)
sub_df['key'] = sub_df.apply(lambda x: func(x), axis=1)
coo_d = merged_train_df.groupby('key')[['mean_rate_case_last7', 'mean_rate_case_each7',
       'mean_rate_fat_last7', 'mean_rate_fat_each7', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case','mean_rate_case_last3','mean_rate_fat_last3',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat','Lat','Long',
       'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities','total_pop', 'smokers_perc', 'density']].mean().reset_index()
loc_group = ['key']
def preprocess(df):
        df["Date"] = df["Date"].astype("datetime64[ms]")
     #   df["days"] = (df["Date"] - pd.to_datetime("2020-01-01")).dt.days
    #    df["weekend"] = df["Date"].dt.dayofweek//5

        df = df.merge(coo_d, how="left", on='key')
        df["Lat"] = (df["Lat"] // 30).astype(np.float32).fillna(0)
        df["Long"] = (df["Long"] // 60).astype(np.float32).fillna(0)
        key_dummies = pd.get_dummies(df['key'])
        df = pd.concat([df,key_dummies],axis=1)
        for col in loc_group:
            df[col].fillna("none", inplace=True)
        return df


#x_train = merged_train_df2
sub_df=preprocess(sub_df)
x_train = preprocess(train_df)
x_test = sub_df
x_train.loc[x_train["Date"]<x_test['Date'].min(),"split"] = "train"
x_train.loc[x_train["Date"]>=x_test['Date'].min(),"split"] = "test"       

featureslist=x_train.columns.unique().tolist()
features=['mean_rate_case_last7',
 'mean_rate_case_each7',
 'mean_rate_fat_last7',
 'mean_rate_fat_each7',
 'max_rate_case',
 'min_rate_case',
 'std_rate_case',
 'mode_rate_case',
 'range_rate_case',
 'mean_rate_case_last3',
 'mean_rate_fat_last3',
 'max_to_min_rate_case',
 'max_rate_fat',
 'min_rate_fat',
 'std_rate_fat',
 'mode_rate_fat',
 'mean_rate_case',
 'mean_rate_fat',
 'range_rate_fat',
 'Lat',
 'Long',
 'max_to_min_rate_fat',
 'rate_ConfirmedCases',
 'rate_Fatalities',
 'total_pop',
 'smokers_perc',
 'density',
 ]
def train_model(df, label, base_label, features=features, **kwargs):
    X_train = df.loc[df["split"] == "train"][features]
    y_train = np.log(df.loc[df["split"] == "train"][label] + 1)
    b_train = np.log(df.loc[df["split"] == "train"][base_label] + 1)
    X_test = df.loc[df["split"] == "test"][features]
    y_test = np.log(df.loc[df["split"] == "test", label] + 1)
    b_test = np.log(df.loc[df["split"] == "test", base_label] + 1)
    print(kwargs)
    model = lgb.LGBMRegressor(**kwargs)
    model.fit(X_train, y_train, init_score = b_train)
    y_pred = model.predict(X_test)
    print(np.sqrt(mean_squared_error(y_test, y_pred + b_test)))
   # print(len(X_test))
    return model







In [ ]:
lgb_model_cases = train_model(x_train,"ConfirmedCases",'cases_prev',features,
                                   max_depth=5,
                                   colsample_bytree=0.8,
                                   learning_rate=0.1,
                                   n_estimators=1000,
                                   subsample=0.8)

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(lgb_model_cases.feature_importances_,x_train[features])), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')



lgb_model_fatalities = train_model(x_train, "Fatalities",'fat_prev',features, 
                                   max_depth=5,
                                   colsample_bytree=0.8,
                                   learning_rate=0.1,
                                   n_estimators=500,
                                   subsample=0.8
                                  )




In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(lgb_model_fatalities.feature_importances_,x_train[features])), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')






In [ ]:
X_train = x_train[features]
y_train = np.log(x_train["ConfirmedCases"] + 1)
b_train = np.log(x_train["cases_prev"] + 1)
cases_model = lgb.LGBMRegressor(max_depth=5,
                                   colsample_bytree=0.8,
                                   learning_rate=0.1,
                                   n_estimators=500,
                                   subsample=0.8
                               )
cases_model.fit(X_train, y_train, init_score = b_train)

X_train = x_train[features]
y_train = np.log(x_train["Fatalities"] + 1)
b_train = np.log(x_train["fat_prev"] + 1)
fatalities_model = lgb.LGBMRegressor(max_depth=5,
                                   colsample_bytree=0.8,
                                   learning_rate=0.1,
                                   n_estimators=500,
                                   subsample=0.8)
fatalities_model.fit(X_train, y_train, init_score = b_train)

In [ ]:


df = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/train.csv")
sub_df = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/test.csv")
df["key"]=train_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)
sub_df["key"]=sub_df[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)


coo_df = merged_train_df.groupby("key")[['mean_rate_case_last7', 'mean_rate_case_each7',
       'mean_rate_fat_last7', 'mean_rate_fat_each7', 'max_rate_case',
       'min_rate_case', 'std_rate_case', 'mode_rate_case', 'range_rate_case',
       'max_to_min_rate_case', 'max_rate_fat', 'min_rate_fat', 'std_rate_fat','mean_rate_case_last3','mean_rate_fat_last3',
       'mode_rate_fat', 'mean_rate_case', 'mean_rate_fat', 'range_rate_fat','Lat','Long',
       'max_to_min_rate_fat', 'rate_ConfirmedCases', 'rate_Fatalities','total_pop', 'smokers_perc', 'density']].mean().reset_index()
   # coo_df = coo_df[coo_df["Country_Region"].notnull()]

loc_group = ["key"]
   # merged_train_df['Date']= pd.to_datetime(merged_train_df['Date']) 
  #  merged_train_df["days"] = (merged_train_df["Date"] - pd.to_datetime("2020-01-01")).dt.days
key_list=df['key'].unique()
def preprocess(df):
    df["Date"] = df["Date"].astype("datetime64[ms]")
  #  df["days"] = (df["Date"] - pd.to_datetime("2020-01-01")).dt.days
    #    df["weekend"] = df["Date"].dt.dayofweek//5

    df = df.merge(coo_df, how="left", on="key")
    df["Lat"] = (df["Lat"] // 30).astype(np.float32).fillna(0)
    df["Long"] = (df["Long"] // 60).astype(np.float32).fillna(0)
    key_dummies = pd.get_dummies(df['key'])
    df = pd.concat([df,key_dummies],axis=1)

    for col in loc_group:
        df[col].fillna("none", inplace=True)
    return df

df = preprocess(df)
sub_df = preprocess(sub_df)


#train_df = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/train.csv")
#test_df = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-4/test.csv")

#train_df['Date']= pd.to_datetime(train_df['Date']) 
country_list = train_df ['Country_Region'].unique()
key_list=df['key'].unique()
#train_df['Date']= pd.to_datetime(train_df['Date']) 


scoring_dates = test_df['Date'].unique()
pred_df = pd.DataFrame(columns=train_df.columns)

sub = []
for date in scoring_dates.tolist():
    print(date)
    start=time.time()
    
    if date < train_df['Date'].max():
            
        new_df = df.loc[df["Date"] < date].copy()
        curr_date_df = sub_df.loc[sub_df["Date"] == date].copy()
        curr_date_df["ConfirmedCases"] = 0
        curr_date_df["Fatalities"] = 0
        new_df = new_df.append(curr_date_df).reset_index(drop=True)
        new_df['cases_prev']=new_df.groupby("key")["ConfirmedCases"].shift()
        new_df['fat_prev']=new_df.groupby("key")["Fatalities"].shift()
        
        
    

        predictions = cases_model.predict(new_df[features]) + np.log(new_df["cases_prev"] + 1)
        new_df["predicted_cases"] = round(np.maximum(np.exp(predictions) - 1, new_df["cases_prev"]))
        predictions = fatalities_model.predict(new_df[features]) + np.log(new_df["fat_prev"] + 1)
        new_df["predicted_fatalities"] = np.maximum(np.exp(predictions) - 1, new_df["fat_prev"])
        new_df["predicted_fatalities"] = round(np.minimum(new_df["predicted_fatalities"], new_df["predicted_cases"]*0.2))
        new_df.loc[new_df["Date"] == date, "ConfirmedCases"] = new_df.loc[new_df["Date"] == date, "predicted_cases"]
        new_df.loc[new_df["Date"] == date, "Fatalities"] = new_df.loc[new_df["Date"] == date, "predicted_fatalities"]
        pred_df = pred_df.append(new_df.loc[new_df["Date"] == date][pred_df.columns.tolist()])
            
        
       
    else:
            #new_df = new_df.copy()
        curr_date_df = sub_df.loc[sub_df["Date"] == date].copy()
        curr_date_df["ConfirmedCases"] = 0
        curr_date_df["Fatalities"] = 0
        new_df = new_df.append(curr_date_df).reset_index(drop=True)
        new_df['cases_prev']=new_df.groupby("key")["ConfirmedCases"].shift()
        new_df['fat_prev']=new_df.groupby("key")["Fatalities"].shift()
        
        predictions = cases_model.predict(new_df[features]) + np.log(new_df["cases_prev"] + 1)
        new_df["predicted_cases"] = round(np.maximum(np.exp(predictions) - 1, new_df["cases_prev"]))
        predictions = fatalities_model.predict(new_df[features]) + np.log(new_df["fat_prev"] + 1)
        new_df["predicted_fatalities"] = np.maximum(np.exp(predictions) - 1, new_df["fat_prev"])
        new_df["predicted_fatalities"] = round(np.minimum(new_df["predicted_fatalities"], new_df["predicted_cases"]*0.2))
        new_df.loc[new_df["Date"] == date, "ConfirmedCases"] = new_df.loc[new_df["Date"] == date, "predicted_cases"]
        new_df.loc[new_df["Date"] == date, "Fatalities"] = new_df.loc[new_df["Date"] == date, "predicted_fatalities"]
        pred_df = pred_df.append(new_df.loc[new_df["Date"] == date][pred_df.columns.tolist()])
            
    print(time.time()-start)  
for key in key_list:
    
    X_forecastId = sub_df.loc[(sub_df['key'] == key), ['ForecastId']]
    X_forecastId = X_forecastId.values.tolist()
X_forecastId = [v[0] for v in X_forecastId]
for j in range(len(sub_df['ForecastId'])):
          
    dic = { 'ForecastId': sub_df['ForecastId'][j], 'ConfirmedCases': pred_df['ConfirmedCases'][j], 'Fatalities': pred_df['Fatalities'][j]}
    sub.append(dic)

submission = pd.DataFrame(sub)
submission[['ForecastId','ConfirmedCases','Fatalities']].to_csv(path_or_buf='submission.csv',index=False) 
from Gaussiandistribution import Gaussian

In [ ]:
len(pred_df['ConfirmedCases']),len(sub_df['ForecastId'])

In [ ]:
range(len(sub_df['ForecastId']))

In [ ]:
len(X_forecastId)

In [ ]:
len(key_list)

In [ ]:
pred_df[pred_df['Country_Region']=='Qatar']